<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:10px 5px'> 
Master Thesis Yannik Haller - Data Aggregation (combination of the topic assignments, sentiment scores, and metadata of the articles to one dataframe and generation of the dataframes required for the regression analyses)
</h1>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
Preparation: load packages and set the appropriate working directory
</h2>
</div>

In [1]:
# Import required baseline packages
import re
import os
import glob
import time
import sys
import pandas as pd
import numpy as np
import datetime

# Change pandas' setting to print out long strings
pd.options.display.max_colwidth = 200

# Import plotting tools
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Set global parameters for plotting
import matplotlib.pylab as pylab
params = {'legend.fontsize': 10,
          'figure.figsize': (8, 6),
          'axes.labelsize': 14,
          'axes.titlesize': 16,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10}
pylab.rcParams.update(params)

# Regression and smoothing tools
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold

# Disable warnings
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
warnings.filterwarnings("ignore", category = FutureWarning)

In [2]:
# Set the appropriate working directory
os.chdir('D:\\Dropbox\\MA_data')

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
1. Read in and aggregate/merge all csv-files containing the labelled and quantified data
</h2>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
1.1 Polarity scores
</h2>
</div>

In [3]:
# Read in the German Vader polarity scores
de_vader_pol = pd.read_csv("Sentiment/Vader/GerVADER/de_vader_polarity_combined.csv", index_col = 0)
# Read in the German TextBlob polarity scores
de_blob_pol = pd.read_csv("Sentiment/TextBlob/de_blob_polarity.csv", index_col = 0)
# Merge the two dataframes to one
de_pol = de_vader_pol.merge(de_blob_pol, how = 'inner', left_index = True, right_index = True, validate = "1:1")
# Take a look at the resulting dataframe
#de_pol

In [4]:
# Read in the French Vader polarity scores
fr_vader_pol = pd.read_csv("Sentiment/Vader/VADER-fr/fr_vader_polarity_combined.csv", index_col = 0)
# Read in the French TextBlob polarity scores
fr_blob_pol = pd.read_csv("Sentiment/TextBlob/fr_blob_polarity.csv", index_col = 0)
# Merge the two dataframes to one
fr_pol = fr_vader_pol.merge(fr_blob_pol, how = 'inner', left_index = True, right_index = True, validate = "1:1")
# Take a look at the resulting dataframe
#fr_pol

In [5]:
# Read in the Italian Naïve polarity scores
it_naive_pol = pd.read_csv("Sentiment/Naive/it_naive_polarity.csv", index_col = 0)
# Rename the column containing the Naïve polarity scores to 'Vader_polarity' for merging purposes
it_pol = it_naive_pol.rename(columns = {'Naive_polarity': 'Vader_polarity'})
# Generate a second column called 'Vader_polarity_adj', which also contains the Naïve polarity scores, for merging purposes
it_pol['Vader_polarity_adj'] = it_pol.Vader_polarity
# Read in the Italian Naïve polarity scores that rely on the extended sentiment lexicon
it_naive_pol_2 = pd.read_csv("Sentiment/Naive/it_naive_polarity_2.csv", index_col = 0)
# Rename the column containing the Naïve polarity scores to 'Vader_polarity_adj_2' for merging purposes
it_naive_pol_2.rename(columns = {'Naive_polarity': 'Vader_polarity_adj_2'}, inplace = True)
# Merge the two dataframes it_pol and it_naive_pol_2
it_pol = it_pol.merge(it_naive_pol_2, how = 'inner', left_index = True, right_index = True, validate = "1:1")
# Generate a fourth column called 'Blob_polarity', which also contains the original Naïve polarity scores, for merging purposes
it_pol['Blob_polarity'] = it_pol.Vader_polarity
# Take a look at the resulting dataframe
#it_pol

In [6]:
# Concatenate the above created dataframes (containing the polarity scores of the German, French and Italian articles)
polarity = pd.concat([de_pol, fr_pol, it_pol])
# Sort the dataframe by the index
polarity.sort_index(inplace = True)
# Take a look at the resulting dataframe
polarity

,Vader_polarity,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity
0,0.9785,0.1582,0.1582,0.124348
1,-0.7772,-0.1196,-0.1996,0.100000
2,0.7561,0.0414,-0.0218,0.163700
3,0.8078,0.0780,0.0780,-0.012364
4,-0.8205,-0.0869,-0.1215,0.207273
...,...,...,...,...
2441178,0.9693,0.1817,0.1817,-0.100000
2441179,-0.7441,-0.0514,-0.0514,0.121429
2441180,0.8391,0.2133,0.1293,0.350000
2441181,-0.8689,-0.5667,-0.5667,-0.433333


In [7]:
# Check for missing values
polarity.isna().sum()

Vader_polarity          0
Vader_polarity_adj      0
Vader_polarity_adj_2    0
Blob_polarity           0
dtype: int64

In [8]:
# Inspect the datatypes of the columns
polarity.dtypes

Vader_polarity          float64
Vader_polarity_adj      float64
Vader_polarity_adj_2    float64
Blob_polarity           float64
dtype: object

In [9]:
# Remove the unnecessary variables to save RAM
del de_pol, de_vader_pol, de_blob_pol, fr_pol, fr_vader_pol, fr_blob_pol, it_pol, it_naive_pol, it_naive_pol_2

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
1.2 Topic assignments
</h2>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h3 style='margin:10px 5px'> 
1.2.1 General topics covered by newspapers (media sections)
</h3>
</div>

In [10]:
# Read in the Dataframe containing the topic assignments and topic affiliation probabilities from the German general LDA model
de_topics = pd.read_csv("LDA/Topic_Assignment/de_topic_assignment.csv", index_col = 0, dtype = {'Topic_ID_1': int, 'Affiliation_Prob_1': float, 'Topic_ID_2': float, 'Affiliation_Prob_2': float})

In [11]:
# Create a column to indicate the main topic for each topic ID (for both, the most and second most likely topic)
de_topics['Topic_1'] = np.repeat(None, de_topics.shape[0])
de_topics['Topic_2'] = np.repeat(None, de_topics.shape[0])
## Assign main topics according to the topic IDs
# Most likely topics
de_topics.loc[de_topics.Topic_ID_1.isin([0]),            'Topic_1'] = 'transportation'
de_topics.loc[de_topics.Topic_ID_1.isin([1]),            'Topic_1'] = 'international_news'
de_topics.loc[de_topics.Topic_ID_1.isin([2,7,12,16,19]), 'Topic_1'] = 'inconsequential'
de_topics.loc[de_topics.Topic_ID_1.isin([3,15,20,26]),   'Topic_1'] = 'sports'
de_topics.loc[de_topics.Topic_ID_1.isin([4,5]),          'Topic_1'] = 'culture'
de_topics.loc[de_topics.Topic_ID_1.isin([6,17,18,21]),   'Topic_1'] = 'economy_national'
de_topics.loc[de_topics.Topic_ID_1.isin([8,11]),         'Topic_1'] = 'public_affairs'
de_topics.loc[de_topics.Topic_ID_1.isin([9]),            'Topic_1'] = 'tourism'
de_topics.loc[de_topics.Topic_ID_1.isin([10,27]),        'Topic_1'] = 'politics_national'
de_topics.loc[de_topics.Topic_ID_1.isin([13,29]),        'Topic_1'] = 'law_order'
de_topics.loc[de_topics.Topic_ID_1.isin([14]),           'Topic_1'] = 'economy_international'
de_topics.loc[de_topics.Topic_ID_1.isin([22]),           'Topic_1'] = 'public_health'
de_topics.loc[de_topics.Topic_ID_1.isin([23]),           'Topic_1'] = 'tragedies_crimes'
de_topics.loc[de_topics.Topic_ID_1.isin([24,25,28]),     'Topic_1'] = 'politics_international'
de_topics.loc[de_topics.Topic_ID_1.isin([30]),           'Topic_1'] = 'COVID'
# Second most likely topics
de_topics.loc[de_topics.Topic_ID_2.isin([0]),            'Topic_2'] = 'transportation'
de_topics.loc[de_topics.Topic_ID_2.isin([1]),            'Topic_2'] = 'international_news'
de_topics.loc[de_topics.Topic_ID_2.isin([2,7,12,16,19]), 'Topic_2'] = 'inconsequential'
de_topics.loc[de_topics.Topic_ID_2.isin([3,15,20,26]),   'Topic_2'] = 'sports'
de_topics.loc[de_topics.Topic_ID_2.isin([4,5]),          'Topic_2'] = 'culture'
de_topics.loc[de_topics.Topic_ID_2.isin([6,17,18,21]),   'Topic_2'] = 'economy_national'
de_topics.loc[de_topics.Topic_ID_2.isin([8,11]),         'Topic_2'] = 'public_affairs'
de_topics.loc[de_topics.Topic_ID_2.isin([9]),            'Topic_2'] = 'tourism'
de_topics.loc[de_topics.Topic_ID_2.isin([10,27]),        'Topic_2'] = 'politics_national'
de_topics.loc[de_topics.Topic_ID_2.isin([13,29]),        'Topic_2'] = 'law_order'
de_topics.loc[de_topics.Topic_ID_2.isin([14]),           'Topic_2'] = 'economy_international'
de_topics.loc[de_topics.Topic_ID_2.isin([22]),           'Topic_2'] = 'public_health'
de_topics.loc[de_topics.Topic_ID_2.isin([23]),           'Topic_2'] = 'tragedies_crimes'
de_topics.loc[de_topics.Topic_ID_2.isin([24,25,28]),     'Topic_2'] = 'politics_international'
de_topics.loc[de_topics.Topic_ID_2.isin([30]),           'Topic_2'] = 'COVID'

In [12]:
# Read in the Dataframe containing the topic assignments and topic affiliation probabilities from the French general LDA model
fr_topics = pd.read_csv("LDA/Topic_Assignment/fr_topic_assignment.csv", index_col = 0, dtype = {'Topic_ID_1': int, 'Affiliation_Prob_1': float, 'Topic_ID_2': float, 'Affiliation_Prob_2': float})

In [13]:
# Create a column to indicate the main topic for each topic ID (for both, the most and second most likely topic)
fr_topics['Topic_1'] = np.repeat(None, fr_topics.shape[0])
fr_topics['Topic_2'] = np.repeat(None, fr_topics.shape[0])
## Assign main topics according to the topic IDs
# Most likely topics
fr_topics.loc[fr_topics.Topic_ID_1.isin([0]),            'Topic_1'] = 'transportation'
fr_topics.loc[fr_topics.Topic_ID_1.isin([1,4,10]),       'Topic_1'] = 'politics_international'
fr_topics.loc[fr_topics.Topic_ID_1.isin([2,24]),         'Topic_1'] = 'politics_national'
fr_topics.loc[fr_topics.Topic_ID_1.isin([3,6,21,23,25]), 'Topic_1'] = 'inconsequential'
fr_topics.loc[fr_topics.Topic_ID_1.isin([5]),            'Topic_1'] = 'law_order'
fr_topics.loc[fr_topics.Topic_ID_1.isin([7]),            'Topic_1'] = 'economy_international'
fr_topics.loc[fr_topics.Topic_ID_1.isin([8,19]),         'Topic_1'] = 'culture'
fr_topics.loc[fr_topics.Topic_ID_1.isin([9,11]),         'Topic_1'] = 'economy_national'
fr_topics.loc[fr_topics.Topic_ID_1.isin([12]),           'Topic_1'] = 'tourism'
fr_topics.loc[fr_topics.Topic_ID_1.isin([13,14,22]),     'Topic_1'] = 'public_affairs'
fr_topics.loc[fr_topics.Topic_ID_1.isin([15]),           'Topic_1'] = 'public_health'
fr_topics.loc[fr_topics.Topic_ID_1.isin([16]),           'Topic_1'] = 'COVID'
fr_topics.loc[fr_topics.Topic_ID_1.isin([17,26]),        'Topic_1'] = 'sports'
fr_topics.loc[fr_topics.Topic_ID_1.isin([18]),           'Topic_1'] = 'international_news'
fr_topics.loc[fr_topics.Topic_ID_1.isin([20]),           'Topic_1'] = 'tragedies_crimes'
# Second most likely topics
fr_topics.loc[fr_topics.Topic_ID_2.isin([0]),            'Topic_2'] = 'transportation'
fr_topics.loc[fr_topics.Topic_ID_2.isin([1,4,10]),       'Topic_2'] = 'politics_international'
fr_topics.loc[fr_topics.Topic_ID_2.isin([2,24]),         'Topic_2'] = 'politics_national'
fr_topics.loc[fr_topics.Topic_ID_2.isin([3,6,21,23,25]), 'Topic_2'] = 'inconsequential'
fr_topics.loc[fr_topics.Topic_ID_2.isin([5]),            'Topic_2'] = 'law_order'
fr_topics.loc[fr_topics.Topic_ID_2.isin([7]),            'Topic_2'] = 'economy_international'
fr_topics.loc[fr_topics.Topic_ID_2.isin([8,19]),         'Topic_2'] = 'culture'
fr_topics.loc[fr_topics.Topic_ID_2.isin([9,11]),         'Topic_2'] = 'economy_national'
fr_topics.loc[fr_topics.Topic_ID_2.isin([12]),           'Topic_2'] = 'tourism'
fr_topics.loc[fr_topics.Topic_ID_2.isin([13,14,22]),     'Topic_2'] = 'public_affairs'
fr_topics.loc[fr_topics.Topic_ID_2.isin([15]),           'Topic_2'] = 'public_health'
fr_topics.loc[fr_topics.Topic_ID_2.isin([16]),           'Topic_2'] = 'COVID'
fr_topics.loc[fr_topics.Topic_ID_2.isin([17,26]),        'Topic_2'] = 'sports'
fr_topics.loc[fr_topics.Topic_ID_2.isin([18]),           'Topic_2'] = 'international_news'
fr_topics.loc[fr_topics.Topic_ID_2.isin([20]),           'Topic_2'] = 'tragedies_crimes'

In [14]:
# Read in the Dataframe containing the topic assignments and topic affiliation probabilities from the Italian general LDA model
it_topics = pd.read_csv("LDA/Topic_Assignment/it_topic_assignment.csv", index_col = 0, dtype = {'Topic_ID_1': int, 'Affiliation_Prob_1': float, 'Topic_ID_2': float, 'Affiliation_Prob_2': float})

In [15]:
# Create a column to indicate the main topic for each topic ID (for both, the most and second most likely topic)
it_topics['Topic_1'] = np.repeat(None, it_topics.shape[0])
it_topics['Topic_2'] = np.repeat(None, it_topics.shape[0])
## Assign main topics according to the topic IDs (note: for Italian artiles there are no topics asigned to the main topics economy or public_health)
# Most likely topics
it_topics.loc[it_topics.Topic_ID_1.isin([0]),                'Topic_1'] = 'politics_international'
it_topics.loc[it_topics.Topic_ID_1.isin([1,18]),             'Topic_1'] = 'sports'
it_topics.loc[it_topics.Topic_ID_1.isin([2,13]),             'Topic_1'] = 'culture'
it_topics.loc[it_topics.Topic_ID_1.isin([3]),                'Topic_1'] = 'transportation'
it_topics.loc[it_topics.Topic_ID_1.isin([4,10,11,12,14,16]), 'Topic_1'] = 'inconsequential'
it_topics.loc[it_topics.Topic_ID_1.isin([5,7]),              'Topic_1'] = 'COVID'
it_topics.loc[it_topics.Topic_ID_1.isin([6,19]),             'Topic_1'] = 'public_affairs'
it_topics.loc[it_topics.Topic_ID_1.isin([8]),                'Topic_1'] = 'politics_national'
it_topics.loc[it_topics.Topic_ID_1.isin([9]),                'Topic_1'] = 'tragedies_crimes'
it_topics.loc[it_topics.Topic_ID_1.isin([15]),               'Topic_1'] = 'tourism'
it_topics.loc[it_topics.Topic_ID_1.isin([17]),               'Topic_1'] = 'law_order'
# Second most likely topics
it_topics.loc[it_topics.Topic_ID_2.isin([0]),                'Topic_2'] = 'politics_international'
it_topics.loc[it_topics.Topic_ID_2.isin([1,18]),             'Topic_2'] = 'sports'
it_topics.loc[it_topics.Topic_ID_2.isin([2,13]),             'Topic_2'] = 'culture'
it_topics.loc[it_topics.Topic_ID_2.isin([3]),                'Topic_2'] = 'transportation'
it_topics.loc[it_topics.Topic_ID_2.isin([4,10,11,12,14,16]), 'Topic_2'] = 'inconsequential'
it_topics.loc[it_topics.Topic_ID_2.isin([5,7]),              'Topic_2'] = 'COVID'
it_topics.loc[it_topics.Topic_ID_2.isin([6,19]),             'Topic_2'] = 'public_affairs'
it_topics.loc[it_topics.Topic_ID_2.isin([8]),                'Topic_2'] = 'politics_national'
it_topics.loc[it_topics.Topic_ID_2.isin([9]),                'Topic_2'] = 'tragedies_crimes'
it_topics.loc[it_topics.Topic_ID_2.isin([15]),               'Topic_2'] = 'tourism'
it_topics.loc[it_topics.Topic_ID_2.isin([17]),               'Topic_2'] = 'law_order'

In [16]:
# Concatenate the above created dataframes (containing the topic assignments of the German, French and Italian articles)
topics = pd.concat([de_topics, fr_topics, it_topics])
# Sort the dataframe by the index
topics.sort_index(inplace = True)
# Take a look at the resulting dataframe
topics

,Topic_ID_1,Affiliation_Prob_1,Topic_ID_2,Affiliation_Prob_2,Topic_1,Topic_2
0,1,0.329300,7.0,0.292654,politics_international,economy_international
1,7,0.462487,11.0,0.269677,economy_international,economy_national
2,7,0.325522,11.0,0.232608,economy_international,economy_national
3,7,0.473720,11.0,0.470850,economy_international,economy_national
4,7,0.598861,11.0,0.281546,economy_international,economy_national
...,...,...,...,...,...,...
2441178,17,0.466716,2.0,0.251594,economy_national,inconsequential
2441179,12,0.315961,21.0,0.274781,inconsequential,economy_national
2441180,26,0.377586,29.0,0.296713,sports,law_order
2441181,23,0.433544,13.0,0.272896,tragedies_crimes,law_order


In [17]:
# Check for missing values
topics.isna().sum()

Topic_ID_1                0
Affiliation_Prob_1        0
Topic_ID_2            26824
Affiliation_Prob_2    26824
Topic_1                   0
Topic_2               26824
dtype: int64

In [18]:
# Inspect the datatypes of the columns
topics.dtypes

Topic_ID_1              int32
Affiliation_Prob_1    float64
Topic_ID_2            float64
Affiliation_Prob_2    float64
Topic_1                object
Topic_2                object
dtype: object

In [19]:
# Remove the unnecessary variables to save RAM
del de_topics, fr_topics, it_topics

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h3 style='margin:10px 5px'> 
1.2.2 COVID-19-specific topics & topic refinement
</h3>
</div>

In [20]:
# Read in the Dataframe containing the topic assignments and topic affiliation probabilities from the German COVID-LDA model
de_topics_fine = pd.read_csv("LDA/Covid/Topic_Assignment/de_topic_assignment.csv", index_col = 0, usecols = [0,1,2], dtype = {'Topic_ID_1': int, 'Affiliation_Prob_1': float})
# Rename the columns as desired
de_topics_fine.rename(columns = {'Topic_ID_1': 'Topic_ID_fine', 'Affiliation_Prob_1': 'Affiliation_Prob_fine'}, inplace = True)

In [21]:
# Create a column to indicate the main topic for each topic ID
de_topics_fine['Topic_fine'] = np.repeat(None, de_topics_fine.shape[0])
# Assign refined (covid) topics according to the topic IDs
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([0]),                'Topic_fine'] = 'COVID_mask_wearing'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([1,2]),              'Topic_fine'] = 'COVID_traveling'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([3]),                'Topic_fine'] = 'COVID_regulations_international'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([4,7,19]),           'Topic_fine'] = 'COVID_situation_social'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([5,11,12,13,14,17]), 'Topic_fine'] = 'COVID_regulations_national'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([6]),                'Topic_fine'] = 'COVID_healthcare_system'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([8]),                'Topic_fine'] = 'COVID_research'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([9]),                'Topic_fine'] = 'COVID_situation_factual_national'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([10]),               'Topic_fine'] = 'COVID_situation_factual_international'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([15]),               'Topic_fine'] = 'COVID_events'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([16]),               'Topic_fine'] = 'COVID_vaccination'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([18]),               'Topic_fine'] = 'COVID_sports'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([20]),               'Topic_fine'] = 'COVID_outbreak_china'
de_topics_fine.loc[de_topics_fine.Topic_ID_fine.isin([21]),               'Topic_fine'] = 'COVID_economy'

In [22]:
# Read in the Dataframe containing the topic assignments and topic affiliation probabilities from the French COVID-LDA model
fr_topics_fine = pd.read_csv("LDA/Covid/Topic_Assignment/fr_topic_assignment.csv", index_col = 0, usecols = [0,1,2], dtype = {'Topic_ID_1': int, 'Affiliation_Prob_1': float})
# Rename the columns as desired
fr_topics_fine.rename(columns = {'Topic_ID_1': 'Topic_ID_fine', 'Affiliation_Prob_1': 'Affiliation_Prob_fine'}, inplace = True)

In [23]:
# Create a column to indicate the main topic for each topic ID
fr_topics_fine['Topic_fine'] = np.repeat(None, fr_topics_fine.shape[0])
# Assign refined (covid) topics according to the topic IDs
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([0]),     'Topic_fine'] = 'international_news'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([1,15]),  'Topic_fine'] = 'COVID_economy'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([2,14]),  'Topic_fine'] = 'COVID_regulations_international'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([3]),     'Topic_fine'] = 'COVID_research'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([4]),     'Topic_fine'] = 'COVID_mask_wearing'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([5]),     'Topic_fine'] = 'COVID_situation_factual_international'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([6]),     'Topic_fine'] = 'COVID_situation_factual_national'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([7]),     'Topic_fine'] = 'COVID_events'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([8]),     'Topic_fine'] = 'COVID_healthcare_system'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([9,10]),  'Topic_fine'] = 'COVID_regulations_national'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([11]),    'Topic_fine'] = 'COVID_sports'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([12,17]), 'Topic_fine'] = 'COVID_traveling'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([13]),    'Topic_fine'] = 'COVID_vaccination'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([16]),    'Topic_fine'] = 'COVID_outbreak_china'
fr_topics_fine.loc[fr_topics_fine.Topic_ID_fine.isin([18]),    'Topic_fine'] = 'COVID_situation_social'

In [24]:
# Read in the Dataframe containing the topic assignments and topic affiliation probabilities from the Italian COVID-LDA model
it_topics_fine = pd.read_csv("LDA/Covid/Topic_Assignment/it_topic_assignment.csv", index_col = 0, usecols = [0,1,2], dtype = {'Topic_ID_1': int, 'Affiliation_Prob_1': float})
# Rename the columns as desired
it_topics_fine.rename(columns = {'Topic_ID_1': 'Topic_ID_fine', 'Affiliation_Prob_1': 'Affiliation_Prob_fine'}, inplace = True)

In [25]:
# Create a column to indicate the main topic for each topic ID
it_topics_fine['Topic_fine'] = np.repeat(None, it_topics_fine.shape[0])
# Assign refined (covid) topics according to the topic IDs
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([0,1,3]), 'Topic_fine'] = 'COVID_economy'
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([2]),     'Topic_fine'] = 'COVID_mask_wearing'
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([4]),     'Topic_fine'] = 'COVID_sports'
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([5,7,9]), 'Topic_fine'] = 'COVID_regulations_national'
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([6]),     'Topic_fine'] = 'COVID_traveling'
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([8]),     'Topic_fine'] = 'COVID_vaccination'
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([10]),    'Topic_fine'] = 'COVID_situation_factual_international'
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([11]),    'Topic_fine'] = 'COVID_healthcare_system'
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([12]),    'Topic_fine'] = 'COVID_situation_factual_national'
it_topics_fine.loc[it_topics_fine.Topic_ID_fine.isin([13]),    'Topic_fine'] = 'public_health'

In [26]:
# Concatenate the above created dataframes (containing the refined topic assignments of the German, French and Italian COVID articles)
topics_fine = pd.concat([de_topics_fine, fr_topics_fine, it_topics_fine])
# Sort the dataframe by the index
topics_fine.sort_index(inplace = True)
# Take a look at the resulting dataframe
topics_fine

,Topic_ID_fine,Affiliation_Prob_fine,Topic_fine
16,18,0.638506,COVID_situation_social
77,15,0.680125,COVID_economy
170,15,0.702011,COVID_economy
198,15,0.316299,COVID_economy
231,10,0.233258,COVID_regulations_national
...,...,...,...
2441147,11,0.523637,COVID_regulations_national
2441158,9,0.936362,COVID_situation_factual_national
2441168,12,0.484031,COVID_regulations_national
2441169,7,0.362894,COVID_situation_social


In [27]:
# Merge the Dataframe containing the general topic assignments with the one that contains the refined COVID topic assigments
topics = topics.merge(topics_fine, how = 'left', left_index = True, right_index = True, validate = "1:1")
# For those articles that are not about COVID, assign the most likely general topic to the 'Topic_fine' variable and the corresponding topic affiliation probability to the 'Affiliation_Prob_fine' variable
topics.loc[topics['Topic_fine'].isna(), 'Affiliation_Prob_fine'] = topics.loc[topics['Topic_fine'].isna(), 'Affiliation_Prob_1']
topics.loc[topics['Topic_fine'].isna(), 'Topic_fine']            = topics.loc[topics['Topic_fine'].isna(), 'Topic_1']
# Take a look at the resulting dataframe
topics

,Topic_ID_1,Affiliation_Prob_1,Topic_ID_2,Affiliation_Prob_2,Topic_1,Topic_2,Topic_ID_fine,Affiliation_Prob_fine,Topic_fine
0,1,0.329300,7.0,0.292654,politics_international,economy_international,NaN,0.329300,politics_international
1,7,0.462487,11.0,0.269677,economy_international,economy_national,NaN,0.462487,economy_international
2,7,0.325522,11.0,0.232608,economy_international,economy_national,NaN,0.325522,economy_international
3,7,0.473720,11.0,0.470850,economy_international,economy_national,NaN,0.473720,economy_international
4,7,0.598861,11.0,0.281546,economy_international,economy_national,NaN,0.598861,economy_international
...,...,...,...,...,...,...,...,...,...
2441178,17,0.466716,2.0,0.251594,economy_national,inconsequential,NaN,0.466716,economy_national
2441179,12,0.315961,21.0,0.274781,inconsequential,economy_national,NaN,0.315961,inconsequential
2441180,26,0.377586,29.0,0.296713,sports,law_order,NaN,0.377586,sports
2441181,23,0.433544,13.0,0.272896,tragedies_crimes,law_order,NaN,0.433544,tragedies_crimes


In [28]:
# Check for missing values
topics.isna().sum()

Topic_ID_1                     0
Affiliation_Prob_1             0
Topic_ID_2                 26824
Affiliation_Prob_2         26824
Topic_1                        0
Topic_2                    26824
Topic_ID_fine            2299496
Affiliation_Prob_fine          0
Topic_fine                     0
dtype: int64

In [29]:
# Inspect the datatypes of the columns
topics.dtypes

Topic_ID_1                 int32
Affiliation_Prob_1       float64
Topic_ID_2               float64
Affiliation_Prob_2       float64
Topic_1                   object
Topic_2                   object
Topic_ID_fine            float64
Affiliation_Prob_fine    float64
Topic_fine                object
dtype: object

In [30]:
# Remove the unnecessary variables to save RAM
del de_topics_fine, fr_topics_fine, it_topics_fine

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2. Read in the metadata of the articles and merge them with the labelled and quantified data (i.e. topic assignments and sentiment scores) to create a dataframe for the main analysis
</h2>
</div>

In [31]:
# Read in the desired metadata of the articles
articles = pd.read_csv("agg_csv.csv", index_col = 0, usecols = [0,1,2,3,4], dtype = {'so': object, 'so_txt': object, 'pubDateTime': object, 'la': object})
# Rename the columns as desired
articles.rename(columns = {'so': 'source_short', 'so_txt': 'source_long', 'pubDateTime': 'publication_date', 'la': 'language'}, inplace = True)
# Drop the entries corresponding to the English articles
articles.drop(articles.loc[articles['language'] == 'en'].index, inplace = True)
# Insert the correct entries to the column 'source_long' whenever it is missing
articles.loc[np.logical_and(articles['source_long'].isna(), articles['source_short'] == 'BLI'), 'source_long'] = 'Blick'
articles.loc[np.logical_and(articles['source_long'].isna(), articles['source_short'] == 'RTS'), 'source_long'] = 'rts.ch'
articles.loc[np.logical_and(articles['source_long'].isna(), articles['source_short'] == 'SRF'), 'source_long'] = 'srf.ch'
articles.loc[np.logical_and(articles['source_long'].isna(), articles['source_short'] == 'TA'), 'source_long']  = 'Tages-Anzeiger'
articles.loc[np.logical_and(articles['source_long'].isna(), articles['source_short'] == 'WEW'), 'source_long'] = 'Die Weltwoche'
articles.loc[np.logical_and(articles['source_long'].isna(), articles['source_short'] == 'WOZ'), 'source_long'] = 'Die Wochenzeitung'
# Unify the entries in the column 'source_long' whenever multiple names are observed for the same source
articles.loc[articles['source_short'] == 'AVU', 'source_long']   = 'Anzeiger von Uster'
articles.loc[articles['source_short'] == 'AGE', 'source_long']   = 'Agefi'
articles.loc[articles['source_short'] == 'BIZO', 'source_long']  = 'Bilanz online'
articles.loc[articles['source_short'] == 'CASO', 'source_long']  = 'Cash Online'
articles.loc[articles['source_short'] == 'FUWO', 'source_long']  = 'Finanz und Wirtschaft Online'
articles.loc[articles['source_short'] == 'INFS', 'source_long']  = 'Infosperber'
articles.loc[articles['source_short'] == 'MEWO', 'source_long']  = 'Medienwoche'
articles.loc[articles['source_short'] == 'NNTA', 'source_long']  = 'Newsnet / Tages-Anzeiger'
articles.loc[articles['source_short'] == 'NNTDG', 'source_long'] = 'Newsnet / Tribune de Genève'
articles.loc[articles['source_short'] == 'NNTLM', 'source_long'] = 'Newsnet / Le Matin'
articles.loc[articles['source_short'] == 'RTS', 'source_long']   = 'rts.ch'
articles.loc[articles['source_short'] == 'RUEM', 'source_long']  = 'Rümlanger'
articles.loc[articles['source_short'] == 'SHZO', 'source_long']  = 'Handelszeitung online'
articles.loc[articles['source_short'] == 'SWII', 'source_long']  = 'swissinfo.ch'
# Transform publication_date into a daily date type
articles.publication_date = pd.to_datetime(articles.publication_date, yearfirst = True).astype('datetime64[D]')
# Add several columns indicating the year, month, week of year, weekday, year-month and year-week combination of the publication date
articles['year']       = articles['publication_date'].dt.strftime('%Y')    # year as decimal number
articles['month']      = articles['publication_date'].dt.strftime('%m')    # month as decimal number (01-12)
articles['week']       = articles['publication_date'].dt.strftime('%W')    # week of year as decimal number (00-53), using Monday as first day of week
articles['weekday']    = articles['publication_date'].dt.day_name()        # weekday as string
articles['year_month'] = articles['publication_date'].dt.strftime('%Y-%m') # year-month combination as string of decimal numbers
articles['year_week']  = articles['publication_date'].dt.strftime('%Y-%W') # year-week combination as string of decimal numbers, using Monday as first day of week
# Take a look at the resulting dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39
...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03
2441179,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03
2441180,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03
2441181,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03


In [32]:
# Merge the articles dataframe with the topics dataframe created above
articles = articles.merge(topics, how = 'inner', left_index = True, right_index = True, validate = "1:1")
# Merge the articles Dataframe with the polarity Dataframe created above
articles = articles.merge(polarity, how = 'inner', left_index = True, right_index = True, validate = "1:1")
# Take a look at the resulting Dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,Affiliation_Prob_2,Topic_1,Topic_2,Topic_ID_fine,Affiliation_Prob_fine,Topic_fine,Vader_polarity,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.292654,politics_international,economy_international,NaN,0.329300,politics_international,0.9785,0.1582,0.1582,0.124348
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.269677,economy_international,economy_national,NaN,0.462487,economy_international,-0.7772,-0.1196,-0.1996,0.100000
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.232608,economy_international,economy_national,NaN,0.325522,economy_international,0.7561,0.0414,-0.0218,0.163700
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.470850,economy_international,economy_national,NaN,0.473720,economy_international,0.8078,0.0780,0.0780,-0.012364
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.281546,economy_international,economy_national,NaN,0.598861,economy_international,-0.8205,-0.0869,-0.1215,0.207273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,0.251594,economy_national,inconsequential,NaN,0.466716,economy_national,0.9693,0.1817,0.1817,-0.100000
2441179,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,0.274781,inconsequential,economy_national,NaN,0.315961,inconsequential,-0.7441,-0.0514,-0.0514,0.121429
2441180,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,0.296713,sports,law_order,NaN,0.377586,sports,0.8391,0.2133,0.1293,0.350000
2441181,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,0.272896,tragedies_crimes,law_order,NaN,0.433544,tragedies_crimes,-0.8689,-0.5667,-0.5667,-0.433333


In [33]:
# Check for missing values
articles.isna().sum()

source_short                   0
source_long                    0
publication_date               0
language                       0
year                           0
month                          0
week                           0
weekday                        0
year_month                     0
year_week                      0
Topic_ID_1                     0
Affiliation_Prob_1             0
Topic_ID_2                 26824
Affiliation_Prob_2         26824
Topic_1                        0
Topic_2                    26824
Topic_ID_fine            2299496
Affiliation_Prob_fine          0
Topic_fine                     0
Vader_polarity                 0
Vader_polarity_adj             0
Vader_polarity_adj_2           0
Blob_polarity                  0
dtype: int64

In [34]:
# Inspect the datatypes of the columns
articles.dtypes

source_short                     object
source_long                      object
publication_date         datetime64[ns]
language                         object
year                             object
month                            object
week                             object
weekday                          object
year_month                       object
year_week                        object
Topic_ID_1                        int32
Affiliation_Prob_1              float64
Topic_ID_2                      float64
Affiliation_Prob_2              float64
Topic_1                          object
Topic_2                          object
Topic_ID_fine                   float64
Affiliation_Prob_fine           float64
Topic_fine                       object
Vader_polarity                  float64
Vader_polarity_adj              float64
Vader_polarity_adj_2            float64
Blob_polarity                   float64
dtype: object

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
3. Read in the data containing the cantonal KOFSPI (i.e. an index that proxies the stringency of the locally implemented containment measures) over time and merge them (using the publication dates as the key variable) with the articles dataframe containing the topic-assignments, sentiment scores and meta-data of the articles
</h2>
</div>

In [35]:
# Read in the Swiss KOFSPI data
stringency = pd.read_csv("Corona_data/Stringency_index_switzerland/kof_data_export.csv")
## Rename the columns as desired
# Get a dictionary containing the desired columnnames
columnnames = {}
columnnames['date'] = 'publication_date'
for i in range(1,len(stringency.columns)):
    columnnames[stringency.columns[i]] = stringency.columns[i][18:20]+'_stridx'
# Rename the columns
stringency.rename(columns = columnnames, inplace = True)
# Transform the column publication_date into a date type
stringency.publication_date = pd.to_datetime(stringency.publication_date, yearfirst = True)
# Take a look at the resulting dataframe
stringency

,publication_date,ag_stridx,ai_stridx,ar_stridx,be_stridx,bl_stridx,bs_stridx,ch_stridx,fr_stridx,ge_stridx,...,sh_stridx,so_stridx,sz_stridx,tg_stridx,ti_stridx,ur_stridx,vd_stridx,vs_stridx,zg_stridx,zh_stridx
0,2020-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2020-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2020-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2020-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2021-04-23,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,...,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332
479,2021-04-24,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,...,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332
480,2021-04-25,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,...,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332
481,2021-04-26,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,...,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332


In [36]:
# Store the index of the articles dataframe in the variable idx
idx = articles.index
# Merge the articles dataframe with the stringency dataframe according to the publication_date
articles = articles.merge(stringency, how = 'left', on = 'publication_date', validate = "m:1")
# Reset the index correctly
articles.set_index(idx, inplace = True)
# Set the stringency index of all cantons to 0 for publication dates before the initiation of the Swiss Stringency Index
articles.loc[articles['publication_date'] < min(stringency['publication_date']), articles.columns[23:]] = 0
# Take a look at the resulting dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,sh_stridx,so_stridx,sz_stridx,tg_stridx,ti_stridx,ur_stridx,vd_stridx,vs_stridx,zg_stridx,zh_stridx
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,33.333332,33.333332,33.333332,33.333332,40.833332,33.333332,48.333332,38.333332,38.333332,38.333332
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,33.333332,33.333332,33.333332,33.333332,40.833332,33.333332,48.333332,38.333332,38.333332,38.333332
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,33.333332,33.333332,33.333332,33.333332,40.833332,33.333332,48.333332,38.333332,38.333332,38.333332
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,33.333332,33.333332,33.333332,33.333332,40.833332,33.333332,48.333332,38.333332,38.333332,38.333332
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,33.333332,33.333332,33.333332,33.333332,40.833332,33.333332,48.333332,38.333332,38.333332,38.333332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
2441179,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
2441180,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
2441181,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000


In [37]:
# Check for missing values
articles.isna().sum()

source_short                   0
source_long                    0
publication_date               0
language                       0
year                           0
month                          0
week                           0
weekday                        0
year_month                     0
year_week                      0
Topic_ID_1                     0
Affiliation_Prob_1             0
Topic_ID_2                 26824
Affiliation_Prob_2         26824
Topic_1                        0
Topic_2                    26824
Topic_ID_fine            2299496
Affiliation_Prob_fine          0
Topic_fine                     0
Vader_polarity                 0
Vader_polarity_adj             0
Vader_polarity_adj_2           0
Blob_polarity                  0
ag_stridx                      0
ai_stridx                      0
ar_stridx                      0
be_stridx                      0
bl_stridx                      0
bs_stridx                      0
ch_stridx                      0
fr_stridx 

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
4. Read in the data containing the cantonal readership by source and merge them (using source_long as the key variable) with the articles dataframe
</h2>
</div>

In [38]:
# Read in the readership data
readership = pd.read_csv("NewsOutlets_metadata/ReadershipByDistrict_2008-2013.csv", sep = ';')
# Remove rows with entries concerning the country LI, since only Switzerland is considered within the analysis
readership = readership.loc[readership['country'] == 'CH']
# Remove rows containing 'CH' in the column 'canton', since those contain the nationally aggregated readership per source (which we do not need here)
readership = readership.loc[np.logical_not(readership['canton'] == 'CH')]
# Remove rows with the entry '(Leer)' in the column newspaper_title, since such entries stand for missing values
readership = readership.loc[np.logical_not(readership['newspaper_title'] == '(Leer)')]
# Remove unneeded columns
readership = readership[['newspaper_title', 'edition', 'canton', 'district_2010', 'year', 'readers_population (in k)']]
# Multiply the column 'readers_population (in k)' by 1000 to get the readers population in proper numbers
readership['readers_population (in k)'] = readership['readers_population (in k)']*1000
# Rename the columns as desired
readership.rename(columns = {'newspaper_title': 'source_long', 'district_2010': 'district', 'readers_population (in k)': 'readers_population'}, inplace = True)
# Take a look at the resulting dataframe
readership

,source_long,edition,canton,district,year,readers_population
0,20minuten,fr,FR,La Broye,2008,5000
1,20minuten,fr,FR,La Broye,2009,5000
2,20minuten,fr,FR,La Broye,2010,5000
3,20minuten,fr,FR,La Broye,2011,6000
4,20minuten,fr,FR,La Broye,2012,6000
...,...,...,...,...,...,...
88650,stgaller_oberland_nachrichten,complete,ZH,Meilen,2013,0
88651,stgaller_oberland_nachrichten,complete,ZH,Pfäffikon,2013,0
88652,stgaller_oberland_nachrichten,complete,ZH,Uster,2013,0
88653,stgaller_oberland_nachrichten,complete,ZH,Winterthur,2013,0


In [39]:
## Unify the entries in the column 'source_long', such that the according labels coincide with the ones in the articles dataframe
# 20 Minuten
readership.loc[np.logical_and(readership['source_long'] == "20minuten", readership['edition'] == 'de'), 'source_long'] = '20 minuten'
readership.loc[np.logical_and(readership['source_long'] == "20minuten", readership['edition'] == 'fr'), 'source_long'] = '20 minutes'
readership.loc[np.logical_and(readership['source_long'] == "20minuten", readership['edition'] == 'it'), 'source_long'] = '20 minuti'
readership.drop(readership.loc[np.logical_and(readership['source_long'] == "20minuten", readership['edition'] == 'complete')].index, inplace = True)
# Migros Magazin
readership.loc[np.logical_and(readership['source_long'] == "migros_magazin", readership['edition'] == 'de'), 'source_long'] = 'Migros-Magazin'
readership.loc[np.logical_and(readership['source_long'] == "migros_magazin", readership['edition'] == 'fr'), 'source_long'] = 'Migros Magazine'
readership.drop(readership.loc[np.logical_and(readership['source_long'] == "migros_magazin", readership['edition'] == 'complete')].index, inplace = True)
# For the remaining source labels, we define a dictionary to replace the source labels from the readership dataframe with the corresponding labels used in the articles dataframe
sourcenames = {'24heures': '24 heures', 'aargauer_zeitung': 'Aargauer Zeitung / MLZ', 'agefi': 'Agefi', 'appenzeller_zeitung': 'Appenzeller Zeitung',
               'basellandschaftliche_zeitung': 'Basellandschaftliche Zeitung / MLZ', 'basler_zeitung': 'Basler Zeitung', 'berner_zeitung': 'Berner Zeitung',
               'bieler_tagblatt': 'Bieler Tagblatt', 'bilanz': 'Bilanz online', 'blick': 'Blick', 'bote_der_urschweiz': 'Bote der Urschweiz', 'cashdaily': 'Cash Online',
               'cooperazione': 'Cooperazione', 'coopration': 'Coopération', 'coopzeitung': 'Coopzeitung', 'das_magazin': 'Das Magazin', 'der_bund': 'Der Bund', 
               'der_landbote': 'Der Landbote', 'die_nordostschweiz': 'Ostschweiz am Sonntag', 'die_suedostschweiz': 'Südostschweiz', 'die_weltwoche': 'Die Weltwoche', 
               'die_wochenzeitung': 'Die Wochenzeitung', 'finanz_und_wirtschaft': 'Finanz und Wirtschaft', 'freiburger_nachrichten': 'Freiburger Nachrichten', 
               'furttaler': 'Furttaler', 'ghi': 'GHI', 'glattaler': 'Glattaler', 'glueckspost': 'Glückspost', 'handelszeitung': 'Handelszeitung', 'la_liberte': 'La Liberté', 
               'le_journal_du_jura': 'Le Journal du Jura', 'le_matin': 'Newsnet / Le Matin', 'le_matin_dimanche': 'Le Matin Dimanche', 'le_nouvelliste': 'Le Nouvelliste', 
               'le_temps': 'Le Temps', 'lillustre': "L'Illustré", 'limmattaler_zeitung': 'Limmattaler Zeitung / MLZ', 'neue_luzerner_zeitung': 'Luzerner Zeitung', 
               'neue_nidwaldner_zeitung': 'Nidwaldner Zeitung', 'neue_obwaldner_zeitung': 'Obwaldner Zeitung', 'neue_urner_zeitung': 'Urner  Zeitung', 
               'neue_zuercher_zeitung': 'Neue Zürcher Zeitung', 'neue_zuger_zeitung': 'Zuger Zeitung', 'nzz_am_sonntag': 'NZZ am Sonntag',
               'obersee_nachrichten': 'Obersee Nachrichten', 'oltner_tagblatt': 'Oltner Tagblatt / MLZ', 'schweizer_familie': 'Schweizer Familie', 
               'schweizer_illustrierte': 'Schweizer Illustrierte', 'solothurner_zeitung': 'Solothurner Zeitung / MLZ', 'sonntagsblick': 'Sonntagsblick', 
               'sonntagszeitung': 'SonntagsZeitung', 'stgaller_tagblatt': 'St. Galler Tagblatt', 'tagblatt_stadt_zuerich': 'Tagblatt der Stadt Zürich', 
               'tagesanzeiger': 'Tages-Anzeiger', 'thurgauer_zeitung': 'Thurgauer Zeitung', 'tribune_de_geneve': 'Tribune de Genève', 'walliser_bote': 'Walliser Bote',
               'werdenberger_obertoggenburger': 'Werdenberger & Obertoggenburger', 'willisauer_boter': 'Willisauer Bote', 'zentralschweiz_am_Sonntag': 'Zentralschweiz am Sonntag',
               'zofinger_tagblatt': 'Zofinger Tagblatt / MLZ', 'zuercher_oberlaender': 'Zürcher Oberländer', 'zuercher_unterlaender': 'Zürcher Unterländer', 
               'zuerichsee_zeitung': 'Zürichsee-Zeitung'}
# Apply the relabeling according to the defined dictionary
readership['source_long'].replace(sourcenames, inplace = True)
# Take a look at the resulting dataframe
readership

,source_long,edition,canton,district,year,readers_population
0,20 minutes,fr,FR,La Broye,2008,5000
1,20 minutes,fr,FR,La Broye,2009,5000
2,20 minutes,fr,FR,La Broye,2010,5000
3,20 minutes,fr,FR,La Broye,2011,6000
4,20 minutes,fr,FR,La Broye,2012,6000
...,...,...,...,...,...,...
88650,stgaller_oberland_nachrichten,complete,ZH,Meilen,2013,0
88651,stgaller_oberland_nachrichten,complete,ZH,Pfäffikon,2013,0
88652,stgaller_oberland_nachrichten,complete,ZH,Uster,2013,0
88653,stgaller_oberland_nachrichten,complete,ZH,Winterthur,2013,0


In [40]:
## Take a look at the sources that are contained in the readership dataframe but not in the articles dataframe
# Get the sources contained in the readership dataframe
readership_sources = np.unique(readership['source_long'])
# Get the sources contained in the articles dataframe
articles_sources = np.unique(articles['source_long'])
# Print out the sources that are contained in the readership dataframe but not in the articles dataframe
for source in readership_sources:
    if not source in articles_sources:
        print(source)

20minuten_weekkombi
20minutenfriday
24hebdo
all
blick_am_abend
blick_blick_am_abend
bz
die_woche
heute_blick
l_express
l_impartial
le_matin_bleu
magazin_sonntagsblick
neue_oberaargauer_zeitung
neue_oltner_zeitung
nzz_amsonntag_zentralschweiz
nzz_business
nzz_executive
nzz_executive_business
nzz_executive_plus
nzz_folio
nzz_weekend
schweiz_am_Sonntag
sonntag_der
stgaller_nachrichten
stgaller_oberland_nachrichten


In [41]:
# Reduce the readership dataframe to solely those entries that contain information on sources considered in the articles data
readership = readership.loc[readership.source_long.isin(articles_sources)]

In [42]:
# Take a look at the different editions of each of the remaining sources
for source in np.unique(readership['source_long']):
    editions = np.unique(readership.loc[readership['source_long'] == source, 'edition'])
    print(source, editions)

20 minuten ['de']
20 minutes ['fr']
20 minuti ['it']
24 heures ['complete' 'lacote' 'lausanne' 'lausanneregion' 'nordvaudoisbroye'
 'regions' 'riviera_chablais']
Aargauer Zeitung / MLZ ['complete']
Agefi ['complete']
Appenzeller Zeitung ['complete']
Basellandschaftliche Zeitung / MLZ ['complete' 'normalauflage']
Basler Zeitung ['complete' 'normalauflage']
Berner Zeitung ['complete']
Bieler Tagblatt ['complete']
Bilanz online ['complete']
Blick ['complete']
Bote der Urschweiz ['complete']
Cash Online ['complete']
Cooperazione ['complete']
Coopzeitung ['complete']
Coopération ['complete']
Das Magazin ['complete']
Der Bund ['complete']
Der Landbote ['complete' 'normalauflage']
Die Weltwoche ['complete']
Die Wochenzeitung ['complete']
Finanz und Wirtschaft ['complete']
Freiburger Nachrichten ['complete' 'normalauflage']
Furttaler ['complete']
GHI ['complete']
Glattaler ['complete']
Glückspost ['complete']
Handelszeitung ['complete']
L'Illustré ['complete']
La Liberté ['complete']
Le Journa

In [43]:
## Apply further cleaning according to the different editions per source, wherever it is required
## Note: Edition 'complete' simply adds up the readership data of all considered editions
# 24 heures (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == '24 heures',                          np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Basellandschaftliche Zeitung (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Basellandschaftliche Zeitung / MLZ', np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Basler Zeitung (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Basler Zeitung',                     np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Der Landbote (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Der Landbote',                       np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Freiburger Nachrichten (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Freiburger Nachrichten',             np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Luzerner Zeitung (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Luzerner Zeitung',                   np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Ostschweiz am Sonntag (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Ostschweiz am Sonntag',              np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Südostschweiz (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Südostschweiz',                      np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Tages-Anzeiger (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Tages-Anzeiger',                     np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Thurgauer Zeitung (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Thurgauer Zeitung',                  np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Zürcher Oberländer (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Zürcher Oberländer',                 np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Zürcher Unterländer (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Zürcher Unterländer',                np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)
# Zürichsee-Zeitung (keep only edition 'complete')
readership.drop(labels = readership.loc[np.logical_and(readership['source_long'] == 'Zürichsee-Zeitung',                  np.logical_not(readership['edition'] == 'complete'))].index, inplace = True)

# Take a look at the resulting dataframe
readership

C:\Users\Hallk\.conda\envs\Master_Thesis_env\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,source_long,edition,canton,district,year,readers_population
0,20 minutes,fr,FR,La Broye,2008,5000
1,20 minutes,fr,FR,La Broye,2009,5000
2,20 minutes,fr,FR,La Broye,2010,5000
3,20 minutes,fr,FR,La Broye,2011,6000
4,20 minutes,fr,FR,La Broye,2012,6000
...,...,...,...,...,...,...
81794,Ostschweiz am Sonntag,complete,ZH,Winterthur,2009,76000
81795,Ostschweiz am Sonntag,complete,ZH,Winterthur,2010,74000
81796,Ostschweiz am Sonntag,complete,ZH,Zürich,2008,0
81797,Ostschweiz am Sonntag,complete,ZH,Zürich,2009,0


In [44]:
# Reduce the readership dataframe to the most recent entry (according to the column year) for each source-district combination
readership.sort_values(['source_long', 'canton', 'district', 'year'], ignore_index = True, inplace = True)
readership = readership.groupby(['source_long', 'canton', 'district']).tail(1)
# Take a look at the dataframe
readership

C:\Users\Hallk\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,source_long,edition,canton,district,year,readers_population
5,20 minuten,de,AG,Aarau,2013,17000
11,20 minuten,de,AG,Baden,2013,32000
17,20 minuten,de,AG,Bremgarten,2013,19000
23,20 minuten,de,AG,Brugg,2013,9000
29,20 minuten,de,AG,Kulm,2013,6000
...,...,...,...,...,...,...
56864,Zürichsee-Zeitung,complete,ZH,Meilen,2013,27000
56870,Zürichsee-Zeitung,complete,ZH,Pfäffikon,2013,0
56876,Zürichsee-Zeitung,complete,ZH,Uster,2013,0
56882,Zürichsee-Zeitung,complete,ZH,Winterthur,2013,0


In [45]:
# Create a copy of the readership dataframe which can be used to create a second main dataframe which suits the requirement of the second main regression (i.e. RM2)
readership_2 = readership.copy()

In [46]:
# Get the aggregated readers population for each source
agg_readers_pop_by_src = readership.groupby(['source_long'])['readers_population'].sum().reset_index()
agg_readers_pop_by_src.rename(columns = {'readers_population': 'tot_readers_population'}, inplace = True)
# Merge the previously created dataframe with the readership dataframe
readership = readership.merge(agg_readers_pop_by_src, how = 'left', on = 'source_long', validate = "m:1")
# Take a look at the created dataframe
readership

,source_long,edition,canton,district,year,readers_population,tot_readers_population
0,20 minuten,de,AG,Aarau,2013,17000,1568000
1,20 minuten,de,AG,Baden,2013,32000,1568000
2,20 minuten,de,AG,Bremgarten,2013,19000,1568000
3,20 minuten,de,AG,Brugg,2013,9000,1568000
4,20 minuten,de,AG,Kulm,2013,6000,1568000
...,...,...,...,...,...,...,...
10138,Zürichsee-Zeitung,complete,ZH,Meilen,2013,27000,104000
10139,Zürichsee-Zeitung,complete,ZH,Pfäffikon,2013,0,104000
10140,Zürichsee-Zeitung,complete,ZH,Uster,2013,0,104000
10141,Zürichsee-Zeitung,complete,ZH,Winterthur,2013,0,104000


In [47]:
## Get the cantonal readers population shares per source
# Calculate the readers population shares for each district in a first step
readership['readers_population_share'] = readership['readers_population']/readership['tot_readers_population']
# Create a datafram containing the cantonal readers population share per source
readership = readership.groupby(['source_long','canton'])['readers_population_share'].sum().reset_index()
# Take a look at the head of the created dataframe
readership.head(26)

,source_long,canton,readers_population_share
0,20 minuten,AG,0.095663
1,20 minuten,AI,0.003189
2,20 minuten,AR,0.008929
3,20 minuten,BE,0.175383
4,20 minuten,BL,0.055485
5,20 minuten,BS,0.037628
6,20 minuten,FR,0.015306
7,20 minuten,GE,0.000000
8,20 minuten,GL,0.005740
9,20 minuten,GR,0.005102


In [48]:
# Take a look at the tail of the created dataframe
readership.tail(26)

,source_long,canton,readers_population_share
1768,Zürichsee-Zeitung,AG,0.000000
1769,Zürichsee-Zeitung,AI,0.000000
1770,Zürichsee-Zeitung,AR,0.000000
1771,Zürichsee-Zeitung,BE,0.000000
1772,Zürichsee-Zeitung,BL,0.000000
1773,Zürichsee-Zeitung,BS,0.000000
1774,Zürichsee-Zeitung,FR,0.000000
1775,Zürichsee-Zeitung,GE,0.000000
1776,Zürichsee-Zeitung,GL,0.000000
1777,Zürichsee-Zeitung,GR,0.000000


In [49]:
# Consturct a dataframe with one row for each source contained in the readership dataframe
readership_new = pd.DataFrame(np.unique(readership['source_long']), columns = ['source_long'])
# Add columns to store the corresponding cantonal readers population share per source
for canton_name in np.unique(readership['canton']):
    readership_new[canton_name+'_rps'] = np.repeat(np.nan, readership_new.shape[0])
# Get the corresponding values to fill the created dataframe
cantonal_rps_colnames = readership_new.columns[1:]
for source in np.unique(readership_new['source_long']):
    for i in range(26):
        readership_new.loc[readership_new['source_long'] == source, cantonal_rps_colnames[i]] = readership.loc[readership['source_long'] == source, 'readers_population_share'].values[i]
# Add rows for missing sources whose readership shares coincide with one of the sources that are already contained
readership_new = readership_new.merge(pd.DataFrame(['20 minuten online', 'Finanz und Wirtschaft Online', 'Handelszeitung online', 'Newsnet / 24 heures', 'Newsnet / Basler Zeitung', 'Newsnet / Berner Zeitung',
                                                    'Newsnet / Der Bund', 'Newsnet / Tages-Anzeiger', 'Newsnet / Tribune de Genève'], columns = ['source_long']),
                                      how = 'outer', on = ['source_long'])
# Get the corresponding values to fill the readership shares in the newly added rows
readership_new.loc[readership_new['source_long'] == '20 minuten online',            cantonal_rps_colnames] = readership_new.loc[
    readership_new['source_long'] == '20 minuten', cantonal_rps_colnames].values
readership_new.loc[readership_new['source_long'] == 'Finanz und Wirtschaft Online', cantonal_rps_colnames] = readership_new.loc[
    readership_new['source_long'] == 'Finanz und Wirtschaft', cantonal_rps_colnames].values
readership_new.loc[readership_new['source_long'] == 'Handelszeitung online',        cantonal_rps_colnames] = readership_new.loc[
    readership_new['source_long'] == 'Handelszeitung', cantonal_rps_colnames].values
readership_new.loc[readership_new['source_long'] == 'Newsnet / 24 heures',          cantonal_rps_colnames] = readership_new.loc[
    readership_new['source_long'] == '24 heures', cantonal_rps_colnames].values
readership_new.loc[readership_new['source_long'] == 'Newsnet / Basler Zeitung',     cantonal_rps_colnames] = readership_new.loc[
    readership_new['source_long'] == 'Basler Zeitung', cantonal_rps_colnames].values
readership_new.loc[readership_new['source_long'] == 'Newsnet / Berner Zeitung',     cantonal_rps_colnames] = readership_new.loc[
    readership_new['source_long'] == 'Berner Zeitung', cantonal_rps_colnames].values
readership_new.loc[readership_new['source_long'] == 'Newsnet / Der Bund',           cantonal_rps_colnames] = readership_new.loc[
    readership_new['source_long'] == 'Der Bund', cantonal_rps_colnames].values
readership_new.loc[readership_new['source_long'] == 'Newsnet / Tages-Anzeiger',     cantonal_rps_colnames] = readership_new.loc[
    readership_new['source_long'] == 'Tages-Anzeiger', cantonal_rps_colnames].values
readership_new.loc[readership_new['source_long'] == 'Newsnet / Tribune de Genève',  cantonal_rps_colnames] = readership_new.loc[
    readership_new['source_long'] == 'Tribune de Genève', cantonal_rps_colnames].values
# Take a look at the dataframe
readership_new

,source_long,AG_rps,AI_rps,AR_rps,BE_rps,BL_rps,BS_rps,FR_rps,GE_rps,GL_rps,...,SH_rps,SO_rps,SZ_rps,TG_rps,TI_rps,UR_rps,VD_rps,VS_rps,ZG_rps,ZH_rps
0,20 minuten,0.095663,0.003189,0.008929,0.175383,0.055485,0.037628,0.015306,0.000000,0.005740,...,0.010842,0.040816,0.024235,0.043367,0.000000,0.006378,0.000000,0.000000,0.021046,0.279337
1,20 minutes,0.000000,0.000000,0.000000,0.026455,0.000000,0.000000,0.089947,0.283951,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.412698,0.109347,0.000000,0.000000
2,20 minuti,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.987805,0.000000,0.000000,0.000000,0.000000,0.000000
3,24 heures,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.019608,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.936275,0.024510,0.000000,0.000000
4,Aargauer Zeitung / MLZ,0.881517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.028436,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Newsnet / Basler Zeitung,0.059783,0.000000,0.000000,0.000000,0.586957,0.277174,0.000000,0.000000,0.000000,...,0.000000,0.076087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
74,Newsnet / Berner Zeitung,0.000000,0.000000,0.000000,0.908397,0.000000,0.000000,0.040712,0.000000,0.000000,...,0.000000,0.050891,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75,Newsnet / Der Bund,0.000000,0.000000,0.000000,0.964187,0.000000,0.000000,0.033058,0.000000,0.000000,...,0.000000,0.002755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
76,Newsnet / Tages-Anzeiger,0.089641,0.000000,0.001992,0.021912,0.005976,0.015936,0.001992,0.000000,0.003984,...,0.011952,0.017928,0.027888,0.029880,0.000000,0.001992,0.000000,0.000000,0.017928,0.661355


In [50]:
# Add a column containing the national readership share by source (the values in this column are defined to be 0 for all sources, for which we have access to the readership data and 1 otherwise)
readership_new['CH_rps'] = np.repeat(0, readership_new.shape[0])
# Get the sources for which the readership data are not available
sources_wo_readership_data = []
sources_w_readership_data = np.unique(readership_new['source_long'])
for source in np.unique(articles['source_long']):
    if source not in sources_w_readership_data:
        sources_wo_readership_data.append(source)
# Take a look at the sources for which the readership data are not available
sources_wo_readership_data

['Anzeiger von Uster',
 'Arcinfo',
 'Badener Tagblatt',
 'Berner Oberländer',
 'Bündner Tagblatt',
 'Infosperber',
 'La Broye',
 'Medienwoche',
 'Rümlanger',
 'Schweizer Bauer',
 'Seetaler Bote',
 'Toggenburger Tagblatt',
 'Volketswiler',
 'rts.ch',
 'srf.ch',
 'swissinfo.ch',
 'watson.ch',
 'zentralplus']

In [51]:
# Get the share of sources for which the readership data are available
print('The readership data are available for ', str(np.round(100*(1-(len(sources_wo_readership_data)/len(np.unique(articles['source_long'])))),2)), '% of the sources contained in the articles data.', sep = '')

The readership data are available for 81.25% of the sources contained in the articles data.


In [52]:
# Add rows for missing sources whose readership data are not available
readership_new = readership_new.merge(pd.DataFrame(sources_wo_readership_data, columns = ['source_long']), how = 'outer', on = ['source_long'])
# Fill the missing values of the newly added rows appropriately
for source in sources_wo_readership_data:
    readership_new.loc[readership_new['source_long'] == source, cantonal_rps_colnames] = 0
    readership_new.loc[readership_new['source_long'] == source, 'CH_rps'] = 1
# Order the columns containing the readership population shares alphabetically
columns_ordered = ['source_long']
for col in np.sort(readership_new.columns[1:]):
    columns_ordered.append(col)
readership_new = readership_new[columns_ordered]
# Check for missing values
readership_new.isna().sum()

source_long    0
AG_rps         0
AI_rps         0
AR_rps         0
BE_rps         0
BL_rps         0
BS_rps         0
CH_rps         0
FR_rps         0
GE_rps         0
GL_rps         0
GR_rps         0
JU_rps         0
LU_rps         0
NE_rps         0
NW_rps         0
OW_rps         0
SG_rps         0
SH_rps         0
SO_rps         0
SZ_rps         0
TG_rps         0
TI_rps         0
UR_rps         0
VD_rps         0
VS_rps         0
ZG_rps         0
ZH_rps         0
dtype: int64

In [53]:
# Take a look at the dataframe
readership_new

,source_long,AG_rps,AI_rps,AR_rps,BE_rps,BL_rps,BS_rps,CH_rps,FR_rps,GE_rps,...,SH_rps,SO_rps,SZ_rps,TG_rps,TI_rps,UR_rps,VD_rps,VS_rps,ZG_rps,ZH_rps
0,20 minuten,0.095663,0.003189,0.008929,0.175383,0.055485,0.037628,0.0,0.015306,0.000000,...,0.010842,0.040816,0.024235,0.043367,0.000000,0.006378,0.000000,0.000000,0.021046,0.279337
1,20 minutes,0.000000,0.000000,0.000000,0.026455,0.000000,0.000000,0.0,0.089947,0.283951,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.412698,0.109347,0.000000,0.000000
2,20 minuti,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.987805,0.000000,0.000000,0.000000,0.000000,0.000000
3,24 heures,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.019608,0.019608,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.936275,0.024510,0.000000,0.000000
4,Aargauer Zeitung / MLZ,0.881517,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.028436,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,rts.ch,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
92,srf.ch,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
93,swissinfo.ch,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
94,watson.ch,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [54]:
# Store the index of the articles dataframe in the variable idx
idx = articles.index
# Merge the readership data with the articles dataframe
articles = articles.merge(readership_new, how = 'left', on = 'source_long', validate = "m:1")
# Reset the index correctly
articles.set_index(idx, inplace = True)
# Check for missing values
articles.isna().sum()[50:]

AG_rps    0
AI_rps    0
AR_rps    0
BE_rps    0
BL_rps    0
BS_rps    0
CH_rps    0
FR_rps    0
GE_rps    0
GL_rps    0
GR_rps    0
JU_rps    0
LU_rps    0
NE_rps    0
NW_rps    0
OW_rps    0
SG_rps    0
SH_rps    0
SO_rps    0
SZ_rps    0
TG_rps    0
TI_rps    0
UR_rps    0
VD_rps    0
VS_rps    0
ZG_rps    0
ZH_rps    0
dtype: int64

In [55]:
# Get the share of articles that are published by sources for which the readership data is available
print('The readership data are available for ', str(np.round(100*(1-np.mean(articles['CH_rps'])),2)), '% of the articles contained in the articles data.', sep = '')

The readership data are available for 86.21% of the articles contained in the articles data.


In [56]:
## Create a column containing the weighted stringency index for each source (i.e. the cantonal stringency index weighted by the cantonal readership population shares)
# Get a numpy array containing the cantonal stringencies
arr_cantonal_stringencies = np.array(articles[articles.columns[23:50]])
# Get a numpy array containing the cantonal readership population shares, which serve as the weights
arr_cantonal_rps = np.array(articles[articles.columns[50:77]])
# Calculate the weighted stringency indexes by means of an element-by-element multiplication of the previously created arrays and a subsequent summation over the resulting columns
weighted_stridx = (arr_cantonal_stringencies * arr_cantonal_rps).sum(1)
# Create the column weighted_stridx and fill in the weighted stringency indexes
articles['weighted_stridx'] = weighted_stridx
# Take a look at the resulting dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,SO_rps,SZ_rps,TG_rps,TI_rps,UR_rps,VD_rps,VS_rps,ZG_rps,ZH_rps,weighted_stridx
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.0,0.000000,0.461538,0.0,0.000000,0.000000,40.256409
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.0,0.000000,0.461538,0.0,0.000000,0.000000,40.256409
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.0,0.000000,0.461538,0.0,0.000000,0.000000,40.256409
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.0,0.000000,0.461538,0.0,0.000000,0.000000,40.256409
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.0,0.000000,0.461538,0.0,0.000000,0.000000,40.256409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,0.040816,0.024235,0.043367,0.0,0.006378,0.000000,0.0,0.021046,0.279337,75.000000
2441179,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,0.040816,0.024235,0.043367,0.0,0.006378,0.000000,0.0,0.021046,0.279337,75.000000
2441180,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,0.040816,0.024235,0.043367,0.0,0.006378,0.000000,0.0,0.021046,0.279337,75.000000
2441181,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,0.040816,0.024235,0.043367,0.0,0.006378,0.000000,0.0,0.021046,0.279337,75.000000


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
5. Read in the data containing the cantonal corona metrics (cases & deaths) and merge them (using source_long as the key variable) with the articles dataframe
</h2>
</div>

In [57]:
# Read in the daily covid cases & deaths data
covid_metrics = pd.read_csv("Corona_data/Cases_switzerland/COVID19_Fallzahlen_CH_total.csv")
# Remove unneeded columns
covid_metrics = covid_metrics[['date', 'abbreviation_canton_and_fl', 'ncumul_tested', 'ncumul_conf', 'ncumul_deceased']]
# Rename the columns as desired
covid_metrics.rename(columns = {'date': 'publication_date', 'abbreviation_canton_and_fl': 'canton', 'ncumul_deceased': 'ncumul_deaths'}, inplace = True)
# Transform publication_date into a daily date type
covid_metrics.publication_date = pd.to_datetime(covid_metrics.publication_date, yearfirst = True).astype('datetime64[D]')
# Remove rows with entries concerning the country Fürstentum Lichtenstein (LI/FL), since only Switzerland is considered within the analysis
covid_metrics = covid_metrics.loc[np.logical_not(covid_metrics['canton'] == 'FL')]
# Check for missing values
covid_metrics.isna().sum()

publication_date        0
canton                  0
ncumul_tested       10869
ncumul_conf           245
ncumul_deaths         616
dtype: int64

In [58]:
# The column ncumul_tested appears to contain mostly missing values (10869 out of 11074). Thus it does not add much informative value and therefore is dropped
covid_metrics.drop('ncumul_tested', axis = 1, inplace = True)

In [59]:
## Missing-value imputation
# First sort the dataframe according to canton and publication date
covid_metrics.sort_values(['canton', 'publication_date'], ignore_index = True, inplace = True)
# For each canton, fill the missing values in the columns ncumul_conf and ncumul_deaths with the most recently preceding non-missing value
for canton in np.unique(covid_metrics['canton']):
    covid_metrics.loc[covid_metrics['canton'] == canton, 'ncumul_conf']   = covid_metrics.loc[covid_metrics['canton'] == canton, 'ncumul_conf'].fillna(method = 'ffill')
    covid_metrics.loc[covid_metrics['canton'] == canton, 'ncumul_deaths'] = covid_metrics.loc[covid_metrics['canton'] == canton, 'ncumul_deaths'].fillna(method = 'ffill')
# Check for missing values
covid_metrics.isna().sum()

publication_date      0
canton                0
ncumul_conf           7
ncumul_deaths       163
dtype: int64

In [60]:
# The remaining missing values appear to occur only before the first instances were reported. Thus, we simply replace them with 0
covid_metrics.fillna(0, inplace = True)
# Check for missing values
covid_metrics.isna().sum()

publication_date    0
canton              0
ncumul_conf         0
ncumul_deaths       0
dtype: int64

In [61]:
# Take a look at the dataframe
covid_metrics

,publication_date,canton,ncumul_conf,ncumul_deaths
0,2020-02-26,AG,1.0,0.0
1,2020-02-27,AG,1.0,0.0
2,2020-02-28,AG,1.0,0.0
3,2020-02-29,AG,3.0,0.0
4,2020-03-01,AG,4.0,0.0
...,...,...,...,...
11069,2021-05-30,ZH,111398.0,1402.0
11070,2021-05-31,ZH,111456.0,1403.0
11071,2021-06-01,ZH,111572.0,1403.0
11072,2021-06-02,ZH,111667.0,1403.0


In [62]:
## Calculate the daily new instances
# Get the number of daily confirmed covid cases
covid_metrics['new_conf'] = covid_metrics.groupby(['canton'])['ncumul_conf'].diff().values
covid_metrics.loc[covid_metrics['new_conf'].isna(), 'new_conf'] = covid_metrics.loc[covid_metrics['new_conf'].isna(), 'ncumul_conf']
# Get the number of daily covid deaths
covid_metrics['new_deaths'] = covid_metrics.groupby(['canton'])['ncumul_deaths'].diff().values
covid_metrics.loc[covid_metrics['new_deaths'].isna(), 'new_deaths'] = covid_metrics.loc[covid_metrics['new_deaths'].isna(), 'ncumul_deaths']
# Take a look at the resulting dataframe
covid_metrics

,publication_date,canton,ncumul_conf,ncumul_deaths,new_conf,new_deaths
0,2020-02-26,AG,1.0,0.0,1.0,0.0
1,2020-02-27,AG,1.0,0.0,0.0,0.0
2,2020-02-28,AG,1.0,0.0,0.0,0.0
3,2020-02-29,AG,3.0,0.0,2.0,0.0
4,2020-03-01,AG,4.0,0.0,1.0,0.0
...,...,...,...,...,...,...
11069,2021-05-30,ZH,111398.0,1402.0,50.0,0.0
11070,2021-05-31,ZH,111456.0,1403.0,58.0,1.0
11071,2021-06-01,ZH,111572.0,1403.0,116.0,0.0
11072,2021-06-02,ZH,111667.0,1403.0,95.0,0.0


In [63]:
# Take a look at some summary statistics
covid_metrics.describe()

,ncumul_conf,ncumul_deaths,new_conf,new_deaths
count,11074.000000,11074.000000,11074.000000,11074.000000
mean,10512.282463,199.956926,61.948438,0.976522
std,17918.994397,282.686850,130.938979,2.401719
min,0.000000,0.000000,0.000000,0.000000
25%,596.500000,17.000000,2.000000,0.000000
50%,2301.500000,71.000000,14.000000,0.000000
75%,11893.500000,282.000000,62.000000,1.000000
max,111749.000000,1403.000000,1642.000000,30.000000


In [64]:
# Aggregate the metrics to get the corresponding values on a national level and add them to the covid_metrics dataframe
national_covid_metrics = covid_metrics.groupby(['publication_date'])['ncumul_conf', 'ncumul_deaths', 'new_conf', 'new_deaths'].sum().reset_index()
national_covid_metrics['canton'] = np.repeat('CH', national_covid_metrics.shape[0])
national_covid_metrics = national_covid_metrics[['publication_date', 'canton', 'ncumul_conf', 'ncumul_deaths', 'new_conf', 'new_deaths']]
covid_metrics = pd.concat([covid_metrics, national_covid_metrics], ignore_index = True)

In [65]:
# Define functions to calculate the 7-day-average of the metrics of interest
def seven_day_avg_conf(data):
    '''
    Takes the covid_metrics DataFrame created above and adds a column containing the 7-day-average of the daily new covid cases
    '''
    temp_array = np.array(data['new_conf'].shift(0).fillna(0))
    for i in range(7)[1:]:
        temp_array = np.vstack([temp_array, np.array(data['new_conf'].shift(i).fillna(0))])
    data['7d_avg_conf'] = temp_array.mean(0)
    return data

def seven_day_avg_deaths(data):
    '''
    Takes the covid_metrics DataFrame created above and adds a column containing the 7-day-average of the daily covid deaths
    '''
    temp_array = np.array(data['new_deaths'].shift(0).fillna(0))
    for i in range(7)[1:]:
        temp_array = np.vstack([temp_array, np.array(data['new_deaths'].shift(i).fillna(0))])
    data['7d_avg_deaths'] = temp_array.mean(0)
    return data

In [66]:
## Apply the previously defined functions for each canton separately
# 7-day-average of daily new covid cases
covid_metrics = covid_metrics.groupby(['canton']).apply(seven_day_avg_conf)
# 7-day-average of daily covid deaths
covid_metrics = covid_metrics.groupby(['canton']).apply(seven_day_avg_deaths)
# Take a look at the resulting dataframe
covid_metrics

,publication_date,canton,ncumul_conf,ncumul_deaths,new_conf,new_deaths,7d_avg_conf,7d_avg_deaths
0,2020-02-26,AG,1.0,0.0,1.0,0.0,0.142857,0.000000
1,2020-02-27,AG,1.0,0.0,0.0,0.0,0.142857,0.000000
2,2020-02-28,AG,1.0,0.0,0.0,0.0,0.142857,0.000000
3,2020-02-29,AG,3.0,0.0,2.0,0.0,0.428571,0.000000
4,2020-03-01,AG,4.0,0.0,1.0,0.0,0.571429,0.000000
...,...,...,...,...,...,...,...,...
11585,2021-05-30,CH,651800.0,10310.0,325.0,0.0,722.285714,3.428571
11586,2021-05-31,CH,684701.0,10812.0,783.0,1.0,766.571429,3.000000
11587,2021-06-01,CH,685345.0,10812.0,644.0,0.0,698.000000,2.285714
11588,2021-06-02,CH,647981.0,10028.0,496.0,0.0,633.857143,2.285714


In [67]:
## Cell 67 ##
# Read in the population data from 2019 (since this is the most recent year for which the population data is available)
population_by_canton = pd.read_csv("Corona_data/CH-Ständige_Wohnbevölkerung_nach_Kanton_2019.csv", sep = ';')
# Merge the covid_metrics dataframe with the population dataframe according to the canton
covid_metrics = covid_metrics.merge(population_by_canton, how = 'left', on = 'canton', validate = "m:1")
## Create columns containing the metrics per million population
# daily new covid cases per million population
covid_metrics['new_conf_pm'] =      covid_metrics['new_conf'].values /      (covid_metrics['canton_population'].values / 10**6)
# daily covid deaths per million population
covid_metrics['new_deaths_pm'] =    covid_metrics['new_deaths'].values /    (covid_metrics['canton_population'].values / 10**6)
# 7-day-average of daily new covid cases per million population
covid_metrics['7d_avg_conf_pm'] =   covid_metrics['7d_avg_conf'].values /   (covid_metrics['canton_population'].values / 10**6)
# 7-day-average of daily covid deaths per million population
covid_metrics['7d_avg_deaths_pm'] = covid_metrics['7d_avg_deaths'].values / (covid_metrics['canton_population'].values / 10**6)
# Take a look at the resulting dataframe
covid_metrics

,publication_date,canton,ncumul_conf,ncumul_deaths,new_conf,new_deaths,7d_avg_conf,7d_avg_deaths,canton_population,new_conf_pm,new_deaths_pm,7d_avg_conf_pm,7d_avg_deaths_pm
0,2020-02-26,AG,1.0,0.0,1.0,0.0,0.142857,0.000000,677102,1.476882,0.0000,0.210983,0.000000
1,2020-02-27,AG,1.0,0.0,0.0,0.0,0.142857,0.000000,677102,0.000000,0.0000,0.210983,0.000000
2,2020-02-28,AG,1.0,0.0,0.0,0.0,0.142857,0.000000,677102,0.000000,0.0000,0.210983,0.000000
3,2020-02-29,AG,3.0,0.0,2.0,0.0,0.428571,0.000000,677102,2.953765,0.0000,0.632950,0.000000
4,2020-03-01,AG,4.0,0.0,1.0,0.0,0.571429,0.000000,677102,1.476882,0.0000,0.843933,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11585,2021-05-30,CH,651800.0,10310.0,325.0,0.0,722.285714,3.428571,8438822,38.512484,0.0000,85.590822,0.406286
11586,2021-05-31,CH,684701.0,10812.0,783.0,1.0,766.571429,3.000000,8438822,92.785462,0.1185,90.838677,0.355500
11587,2021-06-01,CH,685345.0,10812.0,644.0,0.0,698.000000,2.285714,8438822,76.313969,0.0000,82.712966,0.270857
11588,2021-06-02,CH,647981.0,10028.0,496.0,0.0,633.857143,2.285714,8438822,58.775976,0.0000,75.112041,0.270857


In [68]:
## Create a dataframe containing the daily new confirmed cases per million population per canton on each date, which is mergable with the articles dataframe
# Construct a dataframe with one row for each date contained in the covid_metrics dataframe
dconfirmed_pm = pd.DataFrame(np.unique(covid_metrics['publication_date']), columns = ['publication_date'])
# Add columns to store the corresponding cantonal daily new confirmed cases per million population
for canton_name in np.unique(covid_metrics['canton']):
    dconfirmed_pm[canton_name+'_dconf_pm'] = np.repeat(np.nan, dconfirmed_pm.shape[0])
# Get the corresponding values to fill the created dataframe
for canton_name in np.unique(covid_metrics['canton']):
    for pubdate in np.unique(covid_metrics.loc[covid_metrics['canton'] == canton_name, 'publication_date']):
        dconfirmed_pm.loc[dconfirmed_pm['publication_date'] == pubdate, canton_name+'_dconf_pm'] = covid_metrics.loc[np.logical_and(
                                                                                                              covid_metrics['publication_date'] == pubdate, 
                                                                                                              covid_metrics['canton'] == canton_name), 
                                                                                                              'new_conf_pm'].values
# Fill missing values with 0
dconfirmed_pm.fillna(0, inplace = True)
# Take a look at the resulting dataframe
dconfirmed_pm

,publication_date,AG_dconf_pm,AI_dconf_pm,AR_dconf_pm,BE_dconf_pm,BL_dconf_pm,BS_dconf_pm,CH_dconf_pm,FR_dconf_pm,GE_dconf_pm,...,SH_dconf_pm,SO_dconf_pm,SZ_dconf_pm,TG_dconf_pm,TI_dconf_pm,UR_dconf_pm,VD_dconf_pm,VS_dconf_pm,ZG_dconf_pm,ZH_dconf_pm
0,2020-01-05,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.474000,0.000000,8.333681,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-01-06,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2020-01-07,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2020-01-08,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2020-01-09,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2021-05-30,23.630118,0.000000,0.0,28.449195,14.023328,63.283129,38.512484,50.638680,25.001042,...,0.000000,0.000000,202.865475,43.496227,34.691305,0.000000,46.887019,73.771338,15.915457,33.096451
512,2021-05-31,48.737118,1138.016059,0.0,83.385572,31.552488,0.000000,92.785462,98.112443,150.006250,...,444.944320,169.776780,76.074553,76.118397,17.345653,279.064576,130.523322,70.820485,31.830914,38.391883
513,2021-06-01,54.644647,379.338686,0.0,42.183289,59.599143,73.830317,76.313969,120.266866,110.421268,...,49.438258,36.907996,152.149106,47.120912,34.691305,83.719373,68.429703,0.000000,15.915457,76.783766
514,2021-06-02,44.306471,442.561801,0.0,25.506175,49.081647,131.839851,58.775976,110.772113,66.669445,...,49.438258,40.598795,158.488652,68.869026,57.818842,111.625830,45.619802,0.000000,63.661828,62.883257


In [69]:
## Create a dataframe containing the daily covid deaths per million population per canton on each date, which is mergable with the articles dataframe
# Construct a dataframe with one row for each date contained in the covid_metrics dataframe
ddeaths_pm = pd.DataFrame(np.unique(covid_metrics['publication_date']), columns = ['publication_date'])
# Add columns to store the corresponding cantonal daily covid deaths per million population
for canton_name in np.unique(covid_metrics['canton']):
    ddeaths_pm[canton_name+'_ddeaths_pm'] = np.repeat(np.nan, ddeaths_pm.shape[0])
# Get the corresponding values to fill the created dataframe
for canton_name in np.unique(covid_metrics['canton']):
    for pubdate in np.unique(covid_metrics.loc[covid_metrics['canton'] == canton_name, 'publication_date']):
        ddeaths_pm.loc[ddeaths_pm['publication_date'] == pubdate, canton_name+'_ddeaths_pm'] = covid_metrics.loc[np.logical_and(
                                                                                                              covid_metrics['publication_date'] == pubdate, 
                                                                                                              covid_metrics['canton'] == canton_name), 
                                                                                                              'new_deaths_pm'].values
# Fill missing values with 0
ddeaths_pm.fillna(0, inplace = True)
# Take a look at the resulting dataframe
ddeaths_pm

,publication_date,AG_ddeaths_pm,AI_ddeaths_pm,AR_ddeaths_pm,BE_ddeaths_pm,BL_ddeaths_pm,BS_ddeaths_pm,CH_ddeaths_pm,FR_ddeaths_pm,GE_ddeaths_pm,...,SH_ddeaths_pm,SO_ddeaths_pm,SZ_ddeaths_pm,TG_ddeaths_pm,TI_ddeaths_pm,UR_ddeaths_pm,VD_ddeaths_pm,VS_ddeaths_pm,ZG_ddeaths_pm,ZH_ddeaths_pm
0,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,2020-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2020-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,2020-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,2020-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2021-05-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
512,2021-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.1185,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.661929
513,2021-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
514,2021-06-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [70]:
## Create a dataframe containing the 7-day-average of daily new covid cases per million population per canton on each date, which is mergable with the articles dataframe
# Construct a dataframe with one row for each date contained in the covid_metrics dataframe
seven_dconfirmed_pm = pd.DataFrame(np.unique(covid_metrics['publication_date']), columns = ['publication_date'])
# Add columns to store the corresponding cantonal 7-day-average of daily new covid cases per million population
for canton_name in np.unique(covid_metrics['canton']):
    seven_dconfirmed_pm[canton_name+'_7dconf_pm'] = np.repeat(np.nan, seven_dconfirmed_pm.shape[0])
# Get the corresponding values to fill the created dataframe
for canton_name in np.unique(covid_metrics['canton']):
    for pubdate in np.unique(covid_metrics.loc[covid_metrics['canton'] == canton_name, 'publication_date']):
        seven_dconfirmed_pm.loc[seven_dconfirmed_pm['publication_date'] == pubdate, canton_name+'_7dconf_pm'] = covid_metrics.loc[np.logical_and(
                                                                                                                    covid_metrics['publication_date'] == pubdate, 
                                                                                                                    covid_metrics['canton'] == canton_name), 
                                                                                                                    '7d_avg_conf_pm'].values
# Fill missing values with 0
seven_dconfirmed_pm.fillna(0, inplace = True)
# Take a look at the resulting dataframe
seven_dconfirmed_pm

,publication_date,AG_7dconf_pm,AI_7dconf_pm,AR_7dconf_pm,BE_7dconf_pm,BL_7dconf_pm,BS_7dconf_pm,CH_7dconf_pm,FR_7dconf_pm,GE_7dconf_pm,...,SH_7dconf_pm,SO_7dconf_pm,SZ_7dconf_pm,TG_7dconf_pm,TI_7dconf_pm,UR_7dconf_pm,VD_7dconf_pm,VS_7dconf_pm,ZG_7dconf_pm,ZH_7dconf_pm
0,2020-01-05,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.067714,0.000000,1.190526,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-01-06,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.067714,0.000000,1.190526,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2020-01-07,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.067714,0.000000,1.190526,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2020-01-08,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.067714,0.000000,1.190526,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2020-01-09,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.067714,0.000000,1.190526,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2021-05-30,60.552177,0.000000,0.0,75.397374,51.084980,54.242682,85.590822,109.867851,93.753906,...,0.000000,0.000000,140.375663,82.849956,35.104297,0.000000,108.799607,107.073828,57.977737,65.341850
512,2021-05-31,59.286278,921.251095,0.0,78.480538,52.587479,50.475829,90.838677,111.676375,106.552059,...,294.863895,106.505931,139.470014,78.707458,32.626347,255.144755,116.764969,99.907470,57.977737,66.949392
513,2021-06-01,54.644647,903.187348,0.0,66.848601,50.584147,58.009535,82.712966,115.745555,98.218378,...,286.035634,95.433532,146.715209,75.600585,32.213355,235.211571,100.472183,82.202348,51.156826,65.909218
514,2021-06-02,48.948101,704.486132,0.0,57.178678,46.577482,69.310093,75.112041,108.963589,87.206015,...,273.676070,101.233360,153.054756,75.082772,36.343272,187.371929,85.084551,0.000000,47.746371,60.897470


In [71]:
## Create a dataframe containing the 7-day-average of daily covid deaths per million population per canton on each date, which is mergable with the articles dataframe
# Construct a dataframe with one row for each date contained in the covid_metrics dataframe
seven_ddeaths_pm = pd.DataFrame(np.unique(covid_metrics['publication_date']), columns = ['publication_date'])
# Add columns to store the corresponding cantonal 7-day-average of daily covid deaths per million population
for canton_name in np.unique(covid_metrics['canton']):
    seven_ddeaths_pm[canton_name+'_7ddeaths_pm'] = np.repeat(np.nan, seven_ddeaths_pm.shape[0])
# Get the corresponding values to fill the created dataframe
for canton_name in np.unique(covid_metrics['canton']):
    for pubdate in np.unique(covid_metrics.loc[covid_metrics['canton'] == canton_name, 'publication_date']):
        seven_ddeaths_pm.loc[seven_ddeaths_pm['publication_date'] == pubdate, canton_name+'_7ddeaths_pm'] = covid_metrics.loc[np.logical_and(
                                                                                                              covid_metrics['publication_date'] == pubdate, 
                                                                                                              covid_metrics['canton'] == canton_name), 
                                                                                                              '7d_avg_deaths_pm'].values
# Fill missing values with 0
seven_ddeaths_pm.fillna(0, inplace = True)
# Take a look at the resulting dataframe
seven_ddeaths_pm

,publication_date,AG_7ddeaths_pm,AI_7ddeaths_pm,AR_7ddeaths_pm,BE_7ddeaths_pm,BL_7ddeaths_pm,BS_7ddeaths_pm,CH_7ddeaths_pm,FR_7ddeaths_pm,GE_7ddeaths_pm,...,SH_7ddeaths_pm,SO_7ddeaths_pm,SZ_7ddeaths_pm,TG_7ddeaths_pm,TI_7ddeaths_pm,UR_7ddeaths_pm,VD_7ddeaths_pm,VS_7ddeaths_pm,ZG_7ddeaths_pm,ZH_7ddeaths_pm
0,2020-01-05,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
1,2020-01-06,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,2020-01-07,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
3,2020-01-08,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,2020-01-09,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2021-05-30,0.0,0.0,0.0,0.420431,0.0,0.753371,0.406286,0.452131,0.0,...,0.000000,0.000000,0.905649,0.517812,0.412992,0.0,0.724124,0.843101,0.0,0.378245
512,2021-05-31,0.0,0.0,0.0,0.420431,0.0,0.753371,0.355500,0.452131,0.0,...,1.765652,0.527257,0.905649,0.000000,0.412992,0.0,0.724124,0.843101,0.0,0.189123
513,2021-06-01,0.0,0.0,0.0,0.420431,0.0,0.753371,0.270857,0.452131,0.0,...,1.765652,0.000000,0.905649,0.000000,0.412992,0.0,0.362062,0.843101,0.0,0.189123
514,2021-06-02,0.0,0.0,0.0,0.420431,0.0,0.753371,0.270857,0.452131,0.0,...,1.765652,0.000000,0.905649,0.000000,0.412992,0.0,0.362062,0.000000,0.0,0.189123


In [72]:
## Merge the articles dataframe with the dconfirmed_pm dataframe
# Store the index of the articles dataframe in the variable idx
idx = articles.index
# Merge the articles dataframe with the dconfirmed_pm dataframe according to the publication_date
articles = articles.merge(dconfirmed_pm, how = 'left', on = 'publication_date', validate = "m:1")
# Reset the index correctly
articles.set_index(idx, inplace = True)
# Set the number of daily new confirmed cases per million population per canton to 0 for publication dates before the first cases occurred
for col in articles.columns[(articles.shape[1]-27):]:
    articles.loc[articles[col].isna(), col] = 0

## Create a column containing the weighted dconfirmed_pm for each source (i.e. the cantonal daily new confirmed cases per million population weighted by the cantonal readership population shares)
# Get a numpy array containing the cantonal dconfirmed_pm values
arr_dconfirmed_pm = np.array(articles[articles.columns[(articles.shape[1]-27):]])
# Get a numpy array containing the cantonal readership population shares, which serve as the weights
arr_cantonal_rps = np.array(articles[articles.columns[50:77]])
# Calculate the weighted dconfirmed_pm by means of an element-by-element multiplication of the previously created arrays and a subsequent summation over the resulting columns
weighted_dconfirmed_pm = (arr_dconfirmed_pm * arr_cantonal_rps).sum(1)
# Create the column weighted_dconfirmed_pm and fill in the calculated values
articles['weighted_dconfirmed_pm'] = weighted_dconfirmed_pm
# Take a look at the resulting dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,SO_dconf_pm,SZ_dconf_pm,TG_dconf_pm,TI_dconf_pm,UR_dconf_pm,VD_dconf_pm,VS_dconf_pm,ZG_dconf_pm,ZH_dconf_pm,weighted_dconfirmed_pm
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.907996,57.055915,21.748113,14.454711,27.906458,133.057756,47.213657,47.746371,64.207115,146.671235
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.907996,57.055915,21.748113,14.454711,27.906458,133.057756,47.213657,47.746371,64.207115,146.671235
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.907996,57.055915,21.748113,14.454711,27.906458,133.057756,47.213657,47.746371,64.207115,146.671235
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.907996,57.055915,21.748113,14.454711,27.906458,133.057756,47.213657,47.746371,64.207115,146.671235
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.907996,57.055915,21.748113,14.454711,27.906458,133.057756,47.213657,47.746371,64.207115,146.671235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,206.684776,266.260936,213.856448,213.929715,334.877491,261.046645,374.758399,246.689585,295.220343,245.907497
2441179,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,206.684776,266.260936,213.856448,213.929715,334.877491,261.046645,374.758399,246.689585,295.220343,245.907497
2441180,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,206.684776,266.260936,213.856448,213.929715,334.877491,261.046645,374.758399,246.689585,295.220343,245.907497
2441181,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,206.684776,266.260936,213.856448,213.929715,334.877491,261.046645,374.758399,246.689585,295.220343,245.907497


In [73]:
## Merge the articles dataframe with the ddeaths_pm dataframe
# Store the index of the articles dataframe in the variable idx
idx = articles.index
# Merge the articles dataframe with the ddeaths_pm dataframe according to the publication_date
articles = articles.merge(ddeaths_pm, how = 'left', on = 'publication_date', validate = "m:1")
# Reset the index correctly
articles.set_index(idx, inplace = True)
# Set the number of daily covid deaths per million population per canton to 0 for publication dates before the first cases occurred
for col in articles.columns[(articles.shape[1]-27):]:
    articles.loc[articles[col].isna(), col] = 0
    
## Create a column containing the weighted ddeaths_pm for each source (i.e. the cantonal daily covid deaths per million population weighted by the cantonal readership population shares)
# Get a numpy array containing the cantonal ddeaths_pm values
arr_ddeaths_pm = np.array(articles[articles.columns[(articles.shape[1]-27):]])
# Get a numpy array containing the cantonal readership population shares, which serve as the weights
arr_cantonal_rps = np.array(articles[articles.columns[50:77]])
# Calculate the weighted ddeaths_pm by means of an element-by-element multiplication of the previously created arrays and a subsequent summation over the resulting columns
weighted_dconfirmed_pm = (arr_ddeaths_pm * arr_cantonal_rps).sum(1)
# Create the column weighted_ddeaths_pm and fill in the calculated values
articles['weighted_ddeaths_pm'] = weighted_dconfirmed_pm
# Take a look at the resulting dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,SO_ddeaths_pm,SZ_ddeaths_pm,TG_ddeaths_pm,TI_ddeaths_pm,UR_ddeaths_pm,VD_ddeaths_pm,VS_ddeaths_pm,ZG_ddeaths_pm,ZH_ddeaths_pm,weighted_ddeaths_pm
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.0000,0.000000,0.000000,0.000000,0.000000,1.267217,0.000000,0.000000,0.661929,1.706711
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.0000,0.000000,0.000000,0.000000,0.000000,1.267217,0.000000,0.000000,0.661929,1.706711
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.0000,0.000000,0.000000,0.000000,0.000000,1.267217,0.000000,0.000000,0.661929,1.706711
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.0000,0.000000,0.000000,0.000000,0.000000,1.267217,0.000000,0.000000,0.661929,1.706711
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.0000,0.000000,0.000000,0.000000,0.000000,1.267217,0.000000,0.000000,0.661929,1.706711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,3.6908,19.018638,10.874057,11.563768,27.906458,0.000000,5.901707,15.915457,4.633503,5.659186
2441179,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,3.6908,19.018638,10.874057,11.563768,27.906458,0.000000,5.901707,15.915457,4.633503,5.659186
2441180,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,3.6908,19.018638,10.874057,11.563768,27.906458,0.000000,5.901707,15.915457,4.633503,5.659186
2441181,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,3.6908,19.018638,10.874057,11.563768,27.906458,0.000000,5.901707,15.915457,4.633503,5.659186


In [74]:
## Merge the articles dataframe with the seven_dconfirmed_pm dataframe
# Store the index of the articles dataframe in the variable idx
idx = articles.index
# Merge the articles dataframe with the seven_dconfirmed_pm dataframe according to the publication_date
articles = articles.merge(seven_dconfirmed_pm, how = 'left', on = 'publication_date', validate = "m:1")
# Reset the index correctly
articles.set_index(idx, inplace = True)
# Set the 7-day-average of daily new covid cases per million population per canton to 0 for publication dates before the first cases occurred
for col in articles.columns[(articles.shape[1]-27):]:
    articles.loc[articles[col].isna(), col] = 0

## Create a column containing the weighted seven_dconfirmed_pm for each source (i.e. the cantonal 7-day-average of daily new covid cases per million population weighted by the cantonal readership population shares)
# Get a numpy array containing the cantonal seven_dconfirmed_pm values
arr_seven_dconfirmed_pm = np.array(articles[articles.columns[(articles.shape[1]-27):]])
# Get a numpy array containing the cantonal readership population shares, which serve as the weights
arr_cantonal_rps = np.array(articles[articles.columns[50:77]])
# Calculate the weighted seven_dconfirmed_pm by means of an element-by-element multiplication of the previously created arrays and a subsequent summation over the resulting columns
weighted_seven_dconfirmed_pm = (arr_seven_dconfirmed_pm * arr_cantonal_rps).sum(1)
# Create the column weighted_dconfirmed_pm and fill in the calculated values
articles['weighted_7dconfirmed_pm'] = weighted_seven_dconfirmed_pm
# Take a look at the resulting dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,SO_7dconf_pm,SZ_7dconf_pm,TG_7dconf_pm,TI_7dconf_pm,UR_7dconf_pm,VD_7dconf_pm,VS_7dconf_pm,ZG_7dconf_pm,ZH_7dconf_pm,weighted_7dconfirmed_pm
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.380739,26.263834,19.676864,12.389752,7.973274,91.782697,36.253343,26.146822,45.862225,92.683854
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.380739,26.263834,19.676864,12.389752,7.973274,91.782697,36.253343,26.146822,45.862225,92.683854
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.380739,26.263834,19.676864,12.389752,7.973274,91.782697,36.253343,26.146822,45.862225,92.683854
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.380739,26.263834,19.676864,12.389752,7.973274,91.782697,36.253343,26.146822,45.862225,92.683854
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,36.380739,26.263834,19.676864,12.389752,7.973274,91.782697,36.253343,26.146822,45.862225,92.683854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,260.464999,230.940608,190.037086,236.231269,418.596863,259.779428,349.465369,231.910946,228.649196,228.809826
2441179,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,260.464999,230.940608,190.037086,236.231269,418.596863,259.779428,349.465369,231.910946,228.649196,228.809826
2441180,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,260.464999,230.940608,190.037086,236.231269,418.596863,259.779428,349.465369,231.910946,228.649196,228.809826
2441181,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,260.464999,230.940608,190.037086,236.231269,418.596863,259.779428,349.465369,231.910946,228.649196,228.809826


In [75]:
## Merge the articles dataframe with the seven_ddeaths_pm dataframe
# Store the index of the articles dataframe in the variable idx
idx = articles.index
# Merge the articles dataframe with the seven_ddeaths_pm dataframe according to the publication_date
articles = articles.merge(seven_ddeaths_pm, how = 'left', on = 'publication_date', validate = "m:1")
# Reset the index correctly
articles.set_index(idx, inplace = True)
# Set the 7-day-average of daily covid deaths per million population per canton to 0 for publication dates before the first cases occurred
for col in articles.columns[(articles.shape[1]-27):]:
    articles.loc[articles[col].isna(), col] = 0
    
## Create a column containing the weighted seven_ddeaths_pm for each source (i.e. the cantonal 7-day-average of daily covid deaths per million population weighted by the cantonal readership population shares)
# Get a numpy array containing the cantonal seven_ddeaths_pm values
arr_seven_ddeaths_pm = np.array(articles[articles.columns[(articles.shape[1]-27):]])
# Get a numpy array containing the cantonal readership population shares, which serve as the weights
arr_cantonal_rps = np.array(articles[articles.columns[50:77]])
# Calculate the weighted seven_ddeaths_pm by means of an element-by-element multiplication of the previously created arrays and a subsequent summation over the resulting columns
weighted_seven_ddeaths_pm = (arr_seven_ddeaths_pm * arr_cantonal_rps).sum(1)
# Create the column weighted_ddeaths_pm and fill in the calculated values
articles['weighted_7ddeaths_pm'] = weighted_seven_ddeaths_pm
# Take a look at the resulting dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,SO_7ddeaths_pm,SZ_7ddeaths_pm,TG_7ddeaths_pm,TI_7ddeaths_pm,UR_7ddeaths_pm,VD_7ddeaths_pm,VS_7ddeaths_pm,ZG_7ddeaths_pm,ZH_7ddeaths_pm,weighted_7ddeaths_pm
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,5.272571,6.339546,12.427493,14.041719,3.986637,2.172372,9.695662,18.189094,5.484555,6.222238
2441179,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,5.272571,6.339546,12.427493,14.041719,3.986637,2.172372,9.695662,18.189094,5.484555,6.222238
2441180,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,5.272571,6.339546,12.427493,14.041719,3.986637,2.172372,9.695662,18.189094,5.484555,6.222238
2441181,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,5.272571,6.339546,12.427493,14.041719,3.986637,2.172372,9.695662,18.189094,5.484555,6.222238


In [76]:
# Take a look at a sparse version of the articles dataframe
selection = np.arange(23).tolist()
selection.append(77)
selection.append(105)
selection.append(133)
selection.append(161)
selection.append(189)
articles[articles.columns[selection]]

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,Topic_fine,Vader_polarity,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity,weighted_stridx,weighted_dconfirmed_pm,weighted_ddeaths_pm,weighted_7dconfirmed_pm,weighted_7ddeaths_pm
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,politics_international,0.9785,0.1582,0.1582,0.124348,40.256409,146.671235,1.706711,92.683854,0.487632
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,-0.7772,-0.1196,-0.1996,0.100000,40.256409,146.671235,1.706711,92.683854,0.487632
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,0.7561,0.0414,-0.0218,0.163700,40.256409,146.671235,1.706711,92.683854,0.487632
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,0.8078,0.0780,0.0780,-0.012364,40.256409,146.671235,1.706711,92.683854,0.487632
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,-0.8205,-0.0869,-0.1215,0.207273,40.256409,146.671235,1.706711,92.683854,0.487632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,economy_national,0.9693,0.1817,0.1817,-0.100000,75.000000,245.907497,5.659186,228.809826,6.222238
2441179,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,inconsequential,-0.7441,-0.0514,-0.0514,0.121429,75.000000,245.907497,5.659186,228.809826,6.222238
2441180,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,sports,0.8391,0.2133,0.1293,0.350000,75.000000,245.907497,5.659186,228.809826,6.222238
2441181,ZWA,20 minuten,2021-01-22,de,2021,01,03,Friday,2021-01,2021-03,...,tragedies_crimes,-0.8689,-0.5667,-0.5667,-0.433333,75.000000,245.907497,5.659186,228.809826,6.222238


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
6. Export and re-read the resulting dataframe
</h2>
</div>

In [77]:
# Export the articles dataframe as a csv file
articles.to_csv("Analysis/articles_main.csv", encoding = 'utf-8-sig')

In [78]:
## Export a sparse version of the articles dataframe as a csv file
# Define a selection of columns to use
selection = np.arange(23).tolist()
selection.append(77)
selection.append(105)
selection.append(133)
selection.append(161)
selection.append(189)
# Export the sparse articles dataframe
articles[articles.columns[selection]].to_csv("Analysis/articles_main_sparse.csv", encoding = 'utf-8-sig')

In [79]:
# Read in the data as follows
articles = pd.read_csv("Analysis/articles_main.csv", index_col = 0)
# Transform publication_date into a daily date type
articles.publication_date = pd.to_datetime(articles.publication_date, yearfirst = True).astype('datetime64[D]')
# Transform the Topic_ID_2 into an integer type (not possible to read in as integer, due to missing values)
articles['Topic_ID_2'] = articles['Topic_ID_2'].astype("Int32")
# Take a look at the dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,SO_7ddeaths_pm,SZ_7ddeaths_pm,TG_7ddeaths_pm,TI_7ddeaths_pm,UR_7ddeaths_pm,VD_7ddeaths_pm,VS_7ddeaths_pm,ZG_7ddeaths_pm,ZH_7ddeaths_pm,weighted_7ddeaths_pm
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.362062,0.000000,0.000000,0.189123,0.487632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,5.272571,6.339546,12.427493,14.041719,3.986637,2.172372,9.695662,18.189094,5.484555,6.222238
2441179,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,5.272571,6.339546,12.427493,14.041719,3.986637,2.172372,9.695662,18.189094,5.484555,6.222238
2441180,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,5.272571,6.339546,12.427493,14.041719,3.986637,2.172372,9.695662,18.189094,5.484555,6.222238
2441181,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,5.272571,6.339546,12.427493,14.041719,3.986637,2.172372,9.695662,18.189094,5.484555,6.222238


In [80]:
# Inspect the datatypes of the columns
articles.dtypes[:50]

source_short                     object
source_long                      object
publication_date         datetime64[ns]
language                         object
year                              int64
month                             int64
week                              int64
weekday                          object
year_month                       object
year_week                        object
Topic_ID_1                        int64
Affiliation_Prob_1              float64
Topic_ID_2                        Int32
Affiliation_Prob_2              float64
Topic_1                          object
Topic_2                          object
Topic_ID_fine                   float64
Affiliation_Prob_fine           float64
Topic_fine                       object
Vader_polarity                  float64
Vader_polarity_adj              float64
Vader_polarity_adj_2            float64
Blob_polarity                   float64
ag_stridx                       float64
ai_stridx                       float64


In [81]:
# Inspect the datatypes of the columns
articles.dtypes[(articles.shape[1]-28):]

AG_7ddeaths_pm          float64
AI_7ddeaths_pm          float64
AR_7ddeaths_pm          float64
BE_7ddeaths_pm          float64
BL_7ddeaths_pm          float64
BS_7ddeaths_pm          float64
CH_7ddeaths_pm          float64
FR_7ddeaths_pm          float64
GE_7ddeaths_pm          float64
GL_7ddeaths_pm          float64
GR_7ddeaths_pm          float64
JU_7ddeaths_pm          float64
LU_7ddeaths_pm          float64
NE_7ddeaths_pm          float64
NW_7ddeaths_pm          float64
OW_7ddeaths_pm          float64
SG_7ddeaths_pm          float64
SH_7ddeaths_pm          float64
SO_7ddeaths_pm          float64
SZ_7ddeaths_pm          float64
TG_7ddeaths_pm          float64
TI_7ddeaths_pm          float64
UR_7ddeaths_pm          float64
VD_7ddeaths_pm          float64
VS_7ddeaths_pm          float64
ZG_7ddeaths_pm          float64
ZH_7ddeaths_pm          float64
weighted_7ddeaths_pm    float64
dtype: object

In [82]:
# Check for missing values
articles.isna().sum()[:50]

source_short                   0
source_long                    0
publication_date               0
language                       0
year                           0
month                          0
week                           0
weekday                        0
year_month                     0
year_week                      0
Topic_ID_1                     0
Affiliation_Prob_1             0
Topic_ID_2                 26824
Affiliation_Prob_2         26824
Topic_1                        0
Topic_2                    26824
Topic_ID_fine            2299496
Affiliation_Prob_fine          0
Topic_fine                     0
Vader_polarity                 0
Vader_polarity_adj             0
Vader_polarity_adj_2           0
Blob_polarity                  0
ag_stridx                      0
ai_stridx                      0
ar_stridx                      0
be_stridx                      0
bl_stridx                      0
bs_stridx                      0
ch_stridx                      0
fr_stridx 

In [83]:
# Check for missing values
articles.isna().sum()[(articles.shape[1]-28):]

AG_7ddeaths_pm          0
AI_7ddeaths_pm          0
AR_7ddeaths_pm          0
BE_7ddeaths_pm          0
BL_7ddeaths_pm          0
BS_7ddeaths_pm          0
CH_7ddeaths_pm          0
FR_7ddeaths_pm          0
GE_7ddeaths_pm          0
GL_7ddeaths_pm          0
GR_7ddeaths_pm          0
JU_7ddeaths_pm          0
LU_7ddeaths_pm          0
NE_7ddeaths_pm          0
NW_7ddeaths_pm          0
OW_7ddeaths_pm          0
SG_7ddeaths_pm          0
SH_7ddeaths_pm          0
SO_7ddeaths_pm          0
SZ_7ddeaths_pm          0
TG_7ddeaths_pm          0
TI_7ddeaths_pm          0
UR_7ddeaths_pm          0
VD_7ddeaths_pm          0
VS_7ddeaths_pm          0
ZG_7ddeaths_pm          0
ZH_7ddeaths_pm          0
weighted_7ddeaths_pm    0
dtype: int64

In [68]:
## Cell 84 ##
# Read in the sparse data as follows
articles_sparse = pd.read_csv("Analysis/articles_main_sparse.csv", index_col = 0)
# Transform publication_date into a daily date type
articles_sparse.publication_date = pd.to_datetime(articles_sparse.publication_date, yearfirst = True).astype('datetime64[D]')
# Transform the Topic_ID_2 into an integer type (not possible to read in as integer, due to missing values)
articles_sparse['Topic_ID_2'] = articles_sparse['Topic_ID_2'].astype("Int32")
# Take a look at the dataframe
articles_sparse

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,Topic_fine,Vader_polarity,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity,weighted_stridx,weighted_dconfirmed_pm,weighted_ddeaths_pm,weighted_7dconfirmed_pm,weighted_7ddeaths_pm
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,politics_international,0.9785,0.1582,0.1582,0.124348,40.256409,146.671235,1.706711,92.683854,0.487632
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,-0.7772,-0.1196,-0.1996,0.100000,40.256409,146.671235,1.706711,92.683854,0.487632
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,0.7561,0.0414,-0.0218,0.163700,40.256409,146.671235,1.706711,92.683854,0.487632
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,0.8078,0.0780,0.0780,-0.012364,40.256409,146.671235,1.706711,92.683854,0.487632
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,-0.8205,-0.0869,-0.1215,0.207273,40.256409,146.671235,1.706711,92.683854,0.487632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,economy_national,0.9693,0.1817,0.1817,-0.100000,75.000000,245.907497,5.659186,228.809826,6.222238
2441179,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,inconsequential,-0.7441,-0.0514,-0.0514,0.121429,75.000000,245.907497,5.659186,228.809826,6.222238
2441180,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,sports,0.8391,0.2133,0.1293,0.350000,75.000000,245.907497,5.659186,228.809826,6.222238
2441181,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,tragedies_crimes,-0.8689,-0.5667,-0.5667,-0.433333,75.000000,245.907497,5.659186,228.809826,6.222238


In [85]:
# Take a look at some summary statistics
np.round(articles_sparse[articles_sparse.columns[20:]].describe(),3)

,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity,weighted_stridx,weighted_dconfirmed_pm,weighted_ddeaths_pm,weighted_7dconfirmed_pm,weighted_7ddeaths_pm
count,2439096.000,2439096.000,2439096.000,2439096.000,2439096.000,2439096.000,2439096.000,2439096.000
mean,0.113,0.090,0.160,20.514,80.774,1.314,75.060,1.262
std,0.181,0.194,0.361,27.236,220.811,4.197,200.556,3.522
min,-1.000,-1.000,-1.000,0.000,0.000,0.000,0.000,0.000
25%,0.013,-0.025,0.000,0.000,0.000,0.000,0.000,0.000
50%,0.126,0.107,0.137,0.000,0.000,0.000,0.000,0.000
75%,0.229,0.219,0.369,38.303,20.601,0.000,18.757,0.113
max,1.000,1.000,1.000,75.000,2763.983,139.532,2117.844,55.813


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
7. Create a second dataframe containing the (average and total) sentiments the cantons' readerships are exposed to by the newspapers on a daily level (used within RM2)
</h2>
</div>

In [69]:
## Before executing the code chunks of this section, the following steps 2 must be done
## 1. Run everything until (and with) cell 67
## 2. Read in articles_sparse by running cell 84
# Remove the articles dataframe to save ram
del articles

In [70]:
### After appropriately combining the language-specific polarity scores of the VPA, VPAII, and BP into a single variable each, 
### rescale these sentiment measures to have a standard deviation of 1 (such that they are comparable to each other) and store them into the dataframe
## VPA
# Rescale the polarity scores and store the results in the Vader_polarity_adj_rsc variable
articles_sparse['Vader_polarity_adj'] = articles_sparse['Vader_polarity_adj'].copy()/np.std(articles_sparse['Vader_polarity_adj'])
## VPAII
# Rescale the polarity scores and store the results in the Vader_polarity_adj_2_rsc variable
articles_sparse['Vader_polarity_adj_2'] = articles_sparse['Vader_polarity_adj_2'].copy()/np.std(articles_sparse['Vader_polarity_adj_2'])
## BP
# Rescale the language-specific BPs prior to merging them
articles_sparse.loc[articles_sparse['language'] == 'de', 'Blob_polarity'] = (articles_sparse.loc[articles_sparse['language'] == 'de', 'Blob_polarity'].copy()/
                                                                                np.std(articles_sparse.loc[articles_sparse['language'] == 'de', 'Blob_polarity']))
articles_sparse.loc[articles_sparse['language'] == 'fr', 'Blob_polarity'] = (articles_sparse.loc[articles_sparse['language'] == 'fr', 'Blob_polarity'].copy()/
                                                                                np.std(articles_sparse.loc[articles_sparse['language'] == 'fr', 'Blob_polarity']))
articles_sparse.loc[articles_sparse['language'] == 'it', 'Blob_polarity'] = (articles_sparse.loc[articles_sparse['language'] == 'it', 'Blob_polarity'].copy()/
                                                                                np.std(articles_sparse.loc[articles_sparse['language'] == 'it', 'Blob_polarity']))
# Rescale the combined polarity scores again and store the results in the Blob_polarity_rsc variable
articles_sparse['Blob_polarity'] = articles_sparse['Blob_polarity'].copy()/np.std(articles_sparse['Blob_polarity'])

In [71]:
# Create a baseline dataframe to fill with the variables we want to use in the second main regression (i.e. RM2)
dates = []
for i in range(len(np.unique(readership_2['canton']))):
    [dates.append(x) for x in np.unique(articles_sparse['publication_date'])]
senti_exposure = pd.DataFrame(dates, columns = ['publication_date'])
# Create a column containing the canton indicating abbreviation
n_dates = len(np.unique(articles_sparse['publication_date']))
cantons = []
for canton in np.unique(readership_2['canton']):
    [cantons.append(x) for x in np.repeat(canton, n_dates)]
senti_exposure['canton'] = cantons
# Take a look at the created dataframe
senti_exposure

,publication_date,canton
0,2019-01-01,AG
1,2019-01-02,AG
2,2019-01-03,AG
3,2019-01-04,AG
4,2019-01-05,AG
...,...,...
19261,2021-01-18,ZH
19262,2021-01-19,ZH
19263,2021-01-20,ZH
19264,2021-01-21,ZH


In [72]:
# Add a column containing the days elapsed since the first day of the observational period
days_elapsed = []
for i in range(len(np.unique(readership_2['canton']))):
    [days_elapsed.append(x) for x in np.arange(0, len(np.unique(articles_sparse['publication_date'])))]
senti_exposure['days_elapsed'] = days_elapsed
# Add several columns indicating the year, month, week of year, weekday combination of the publication date
senti_exposure['year']    = senti_exposure['publication_date'].dt.strftime('%Y').astype(int) # year as decimal number
senti_exposure['month']   = senti_exposure['publication_date'].dt.strftime('%m').astype(int) # month as decimal number (01-12)
senti_exposure['week']    = senti_exposure['publication_date'].dt.strftime('%W').astype(int) # week of year as decimal number (00-53), using Monday as first day of week
senti_exposure['weekday'] = senti_exposure['publication_date'].dt.strftime('%u').astype(int) # weekday as decimal number (1-7, Moday is 1)
# Take a look at the resulting dataframe
senti_exposure

,publication_date,canton,days_elapsed,year,month,week,weekday
0,2019-01-01,AG,0,2019,1,0,2
1,2019-01-02,AG,1,2019,1,0,3
2,2019-01-03,AG,2,2019,1,0,4
3,2019-01-04,AG,3,2019,1,0,5
4,2019-01-05,AG,4,2019,1,0,6
...,...,...,...,...,...,...,...
19261,2021-01-18,ZH,736,2021,1,3,1
19262,2021-01-19,ZH,737,2021,1,3,2
19263,2021-01-20,ZH,738,2021,1,3,3
19264,2021-01-21,ZH,739,2021,1,3,4


In [73]:
# Read in the Swiss Stringency Index (KOFSPI) data again
stringency = pd.read_csv("Corona_data/Stringency_index_switzerland/kof_data_export.csv")
## Rename the columns as desired
# Get a dictionary containing the desired columnnames
columnnames = {}
columnnames['date'] = 'publication_date'
for i in range(1,len(stringency.columns)):
    columnnames[stringency.columns[i]] = stringency.columns[i][18:20]
# Rename the columns
stringency.rename(columns = columnnames, inplace = True)
# Transform the column publication_date into a date type
stringency.publication_date = pd.to_datetime(stringency.publication_date, yearfirst = True)
# Take a look at the resulting dataframe
stringency

,publication_date,ag,ai,ar,be,bl,bs,ch,fr,ge,...,sh,so,sz,tg,ti,ur,vd,vs,zg,zh
0,2020-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2020-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2020-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2020-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2021-04-23,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,...,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332
479,2021-04-24,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,...,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332
480,2021-04-25,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,...,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332
481,2021-04-26,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,...,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332,55.833332


In [74]:
## Add the stringency index data to the senti_exposure dataframe
# Create a column containing missing values as a placeholder
senti_exposure['stridx'] = np.repeat(np.nan, senti_exposure.shape[0])
# Fill the corresponding values
for canton in np.unique(senti_exposure['canton']):
    for date in np.unique(stringency['publication_date']):
        senti_exposure.loc[np.logical_and(senti_exposure['canton'] == canton, senti_exposure['publication_date'] == date), 'stridx'] = \
            stringency.loc[stringency['publication_date'] == date, canton.lower()].values[0]
# Fill the missing values with 0, since those occur on days before the stringency index was introduced due to the corona outbreak
senti_exposure.fillna(0, inplace = True)
# Take a look at the resulting dataframe
senti_exposure

,publication_date,canton,days_elapsed,year,month,week,weekday,stridx
0,2019-01-01,AG,0,2019,1,0,2,0.0
1,2019-01-02,AG,1,2019,1,0,3,0.0
2,2019-01-03,AG,2,2019,1,0,4,0.0
3,2019-01-04,AG,3,2019,1,0,5,0.0
4,2019-01-05,AG,4,2019,1,0,6,0.0
...,...,...,...,...,...,...,...,...
19261,2021-01-18,ZH,736,2021,1,3,1,75.0
19262,2021-01-19,ZH,737,2021,1,3,2,75.0
19263,2021-01-20,ZH,738,2021,1,3,3,75.0
19264,2021-01-21,ZH,739,2021,1,3,4,75.0


In [75]:
# Merge the senti_exposure dataframe with the desired columns of the covid_metrics dataframe
senti_exposure = senti_exposure.merge(covid_metrics[covid_metrics.columns[[0,1,4,5,6,7,9,10,11,12]]], how = 'left', on = ['publication_date', 'canton'], validate = "1:1")
# Merge the senti_exposure dataframe with the population_by_canton dataframe
senti_exposure = senti_exposure.merge(population_by_canton, how = 'left', on = 'canton', validate = "m:1")
# Fill the missing values with 0, since those occur on days before any corona metrics were recorded
senti_exposure.fillna(0, inplace = True)
# Take a look at the resulting dataframe
senti_exposure

,publication_date,canton,days_elapsed,year,month,week,weekday,stridx,new_conf,new_deaths,7d_avg_conf,7d_avg_deaths,new_conf_pm,new_deaths_pm,7d_avg_conf_pm,7d_avg_deaths_pm,canton_population
0,2019-01-01,AG,0,2019,1,0,2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,677102
1,2019-01-02,AG,1,2019,1,0,3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,677102
2,2019-01-03,AG,2,2019,1,0,4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,677102
3,2019-01-04,AG,3,2019,1,0,5,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,677102
4,2019-01-05,AG,4,2019,1,0,6,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,677102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19261,2021-01-18,ZH,736,2021,1,3,1,75.0,210.0,8.0,380.000000,8.428571,139.005094,5.295432,251.533028,5.579116,1510736
19262,2021-01-19,ZH,737,2021,1,3,2,75.0,426.0,15.0,357.000000,9.285714,281.981763,9.928935,236.308660,6.146484,1510736
19263,2021-01-20,ZH,738,2021,1,3,3,75.0,454.0,8.0,348.000000,9.000000,300.515775,5.295432,230.351299,5.957361,1510736
19264,2021-01-21,ZH,739,2021,1,3,4,75.0,316.0,7.0,333.000000,8.857143,209.169570,4.633503,220.422364,5.862800,1510736


In [76]:
### Calculate the sentiment exposed to the cantonal readership on each day and add the values to the senti_exposure dataframe
### Note: we define the daily cantonal sentiment exposure as the weighted average among each (on that day) publishing news outlet's mean/total sentiment among articles published on that day, 
### weighted by the source's readership in the canton under consideration

## Reduce the articles dataframe to those articles, for which the source's cantonal readership data are available
# Define the set of news outlets for which the readership data is not available
sources_wo_readership_data = ['AVU', 'ARC', 'BT', 'BEOL', 'BUET', 'INFS', 'LBH', 'MEWO', 'RUEM', 
                              'SBAU', 'SEBO', 'TBT', 'VOLK', 'RTS', 'SRF', 'SWII', 'WASO', 'ZPLU']
# Remove articles for which the readership data is not available from the articles_sparse dataframe
articles_sparse = articles_sparse.loc[np.logical_not(articles_sparse['source_short'].isin(sources_wo_readership_data))]

## Compute the daily average sentiment for each source and store the results in the avg_sentiments dataframe (retreived sentiment scores: adjusted Vader polarity, adjusted Vader polarity 2, TextBlob polarity)
avg_sentiments = articles_sparse.groupby(['publication_date', 'source_long'])['Vader_polarity_adj', 'Vader_polarity_adj_2', 'Blob_polarity'].mean().reset_index()
## Compute the daily total sentiment for each source and store the results in the tot_sentiments dataframe (retreived sentiment scores: adjusted Vader polarity, adjusted Vader polarity 2, TextBlob polarity)
tot_sentiments = articles_sparse.groupby(['publication_date', 'source_long'])['Vader_polarity_adj', 'Vader_polarity_adj_2', 'Blob_polarity'].sum().reset_index()

## Compute the daily average sentiment among the articles on all general topics (i.e. media sections) for each source and store the results in the avg_sentiments_bytopic dataframe 
## (retreived sentiment scores: adjusted Vader polarity, adjusted Vader polarity 2, TextBlob polarity)
avg_sentiments_bytopic = articles_sparse.groupby(['publication_date', 'source_long', 'Topic_1'])['Vader_polarity_adj', 'Vader_polarity_adj_2', 'Blob_polarity'].mean().reset_index()
## Compute the daily total sentiment among the articles on all general topics (i.e. media sections) for each source and store the results in the tot_sentiments_bytopic dataframe 
## (retreived sentiment scores: adjusted Vader polarity, adjusted Vader polarity 2, TextBlob polarity)
tot_sentiments_bytopic = articles_sparse.groupby(['publication_date', 'source_long', 'Topic_1'])['Vader_polarity_adj', 'Vader_polarity_adj_2', 'Blob_polarity'].sum().reset_index()

## Adjust the readership_2 dataframe such that all news outlets for which the readership data is available are contained appropriately
# Define the set of sources whose readership data is assumed to coincide with other sources that are not yet contained in the readership_2 dataframe
sources_special_cases = ['20 minuten', 'Finanz und Wirtschaft', '24 heures', 'Basler Zeitung', 'Berner Zeitung', 'Der Bund', 'Handelszeitung', 'Tages-Anzeiger', 'Tribune de Genève']
# Retrieve the readership data of sources for which the readership data coincide with other sources already contained in the readership_2 dataframe and add them to the readership_2 dataframe
readership_temp = readership_2[readership_2['source_long'].isin(sources_special_cases)].copy()
readership_temp['source_long'].replace(sources_special_cases, ['20 minuten online', 'Finanz und Wirtschaft Online', 'Newsnet / 24 heures', 'Newsnet / Basler Zeitung',
                                       'Newsnet / Berner Zeitung', 'Newsnet / Der Bund', 'Handelszeitung online', 'Newsnet / Tages-Anzeiger', 'Newsnet / Tribune de Genève'], inplace = True)
readership_2 = pd.concat([readership_2, readership_temp])
# Calculate the cantonal readership for each news outlet and store it within the readership_2 dataframe
readership_use = readership_2.groupby(['source_long', 'canton'])['readers_population'].sum().reset_index()

In [77]:
# Define a function to calculate the (average or total) polarity a canton's readership is exposed to on a specific date
def get_exposed_pol(sentiments, readership, canton, date):
    ## First get the set of newspapers that published at least one article on the date under consideration
    publishing_sources = np.unique(sentiments.loc[sentiments['publication_date'] == date, 'source_long'])
    ## Compute the weights for each source according to the cantonal readership
    weights = readership.loc[np.logical_and(readership['source_long'].isin(publishing_sources), readership['canton'] == canton), 'readers_population'].values
    # If no readership is reached by the sources publishing on date in canton, then return NA
    total_readership = weights.sum()
    if total_readership == 0:
        return np.repeat(np.nan, 3)
    # Otherwise compute the weighted polarity score among the publishing sources on that day (i.e. the average/total sentiment the canton's readership is exposed to on that date)
    else:
        # Normalize the weights, such that they sum up to 1
        weights = weights / total_readership
        # For each news outlet publishing on date, get the average/total polarity for the articles they published on date
        polarity_scores = sentiments.loc[np.logical_and(sentiments['source_long'].isin(publishing_sources), sentiments['publication_date'] == date), 
                                        ['Vader_polarity_adj', 'Vader_polarity_adj_2', 'Blob_polarity']].values
        # Calculate the weighted polarity scores the canton's readership is exposed to on date (three values are returned - one for each polarity type 
        ## --> 'Vader_polarity_adj', 'Vader_polarity_adj_2', 'Blob_polarity')
        return (weights @ polarity_scores)

In [78]:
# Define a function to calcualte the topic-specific (respectively media section-specific) sentiment exposure (as weighted total or average polarity scores) for each canton on each date and add it to the senti_exposure dataframe
def get_topic_specific_exposed_pol(sentiments_bytopic, topic, sentiment_type = 'avg'):
    # Create three columns containing missing values as placeholders to fill in the exposed average sentiments based on the three different polarity scores of interest
    senti_exposure[sentiment_type+'_Vader_polarity_adj_'+topic]   = np.repeat(np.nan, senti_exposure.shape[0])
    senti_exposure[sentiment_type+'_Vader_polarity_adj_2_'+topic] = np.repeat(np.nan, senti_exposure.shape[0])
    senti_exposure[sentiment_type+'_Blob_polarity_'+topic]        = np.repeat(np.nan, senti_exposure.shape[0])
    # Set up a loop to go through each canton and each date within the considered timeframe and add the according topic-specific exposed sentiments to the senti_exposure dataframe
    for canton in np.unique(senti_exposure['canton']):
        for date in np.unique(senti_exposure['publication_date']):
            senti_exposure.loc[np.logical_and(senti_exposure['publication_date'] == date, senti_exposure['canton'] == canton),  
                              [sentiment_type+'_Vader_polarity_adj_'+topic, sentiment_type+'_Vader_polarity_adj_2_'+topic, sentiment_type+'_Blob_polarity_'+topic]] = \
                                  get_exposed_pol(sentiments_bytopic.loc[sentiments_bytopic['Topic_1'] == topic], readership_use, canton, date)

In [79]:
### Weighted AVERAGE polarity scores exposed to cantonal readership
## Calcualte the sentiment exposure for each canton on each date and add it to the senti_exposure dataframe

## Overall
# Create three columns containing missing values as placeholders to fill in the exposed average sentiments based on the three different polarity scores of interest
senti_exposure['avg_Vader_polarity_adj']   = np.repeat(np.nan, senti_exposure.shape[0])
senti_exposure['avg_Vader_polarity_adj_2'] = np.repeat(np.nan, senti_exposure.shape[0])
senti_exposure['avg_Blob_polarity']        = np.repeat(np.nan, senti_exposure.shape[0])
# Set up a loop to go through each canton and each date within the considered timeframe and add the according exposed sentiments to the senti_exposure dataframe
for canton in np.unique(senti_exposure['canton']):
    for date in np.unique(senti_exposure['publication_date']):
        senti_exposure.loc[np.logical_and(senti_exposure['publication_date'] == date, senti_exposure['canton'] == canton),  
                          ['avg_Vader_polarity_adj', 'avg_Vader_polarity_adj_2', 'avg_Blob_polarity']] = get_exposed_pol(avg_sentiments, readership_use, canton, date)

## Topic specific (respectively media section-specific)
# Apply the above defined function to compute the topic-specific average polarity scores exposed to the readerships of all cantons on each date for each topic
for topic in np.unique(avg_sentiments_bytopic['Topic_1']):
    get_topic_specific_exposed_pol(avg_sentiments_bytopic, topic, sentiment_type = 'avg')

In [80]:
### Weighted TOTAL polarity scores exposed to cantonal readership
## Calcualte the sentiment exposure for each canton on each date and add it to the senti_exposure dataframe

## Overall
# Create three columns containing missing values as placeholders to fill in the exposed average sentiments based on the three different polarity scores of interest
senti_exposure['tot_Vader_polarity_adj']   = np.repeat(np.nan, senti_exposure.shape[0])
senti_exposure['tot_Vader_polarity_adj_2'] = np.repeat(np.nan, senti_exposure.shape[0])
senti_exposure['tot_Blob_polarity']        = np.repeat(np.nan, senti_exposure.shape[0])
# Set up a loop to go through each canton and each date within the considered timeframe and add the according exposed sentiments to the senti_exposure dataframe
for canton in np.unique(senti_exposure['canton']):
    for date in np.unique(senti_exposure['publication_date']):
        senti_exposure.loc[np.logical_and(senti_exposure['publication_date'] == date, senti_exposure['canton'] == canton),
                          ['tot_Vader_polarity_adj', 'tot_Vader_polarity_adj_2', 'tot_Blob_polarity']] = get_exposed_pol(tot_sentiments, readership_use, canton, date)

## Topic specific (respectively media section-specific)
# Apply the above defined function to compute the topic-specific total polarity scores exposed to the readerships of all cantons on each date for each topic
for topic in np.unique(tot_sentiments_bytopic['Topic_1']):
    get_topic_specific_exposed_pol(tot_sentiments_bytopic, topic, sentiment_type = 'tot')

In [81]:
# Take a look at the resulting dataframe
senti_exposure

,publication_date,canton,days_elapsed,year,month,week,weekday,stridx,new_conf,new_deaths,...,tot_Blob_polarity_sports,tot_Vader_polarity_adj_tourism,tot_Vader_polarity_adj_2_tourism,tot_Blob_polarity_tourism,tot_Vader_polarity_adj_tragedies_crimes,tot_Vader_polarity_adj_2_tragedies_crimes,tot_Blob_polarity_tragedies_crimes,tot_Vader_polarity_adj_transportation,tot_Vader_polarity_adj_2_transportation,tot_Blob_polarity_transportation
0,2019-01-01,AG,0,2019,1,0,2,0.0,0.0,0.0,...,5.090300,3.591609,3.339084,1.780409,-3.958509,-3.858086,-6.088240,NaN,NaN,NaN
1,2019-01-02,AG,1,2019,1,0,3,0.0,0.0,0.0,...,6.541127,3.561153,3.283226,2.673688,-3.231048,-3.046774,-2.030495,NaN,NaN,NaN
2,2019-01-03,AG,2,2019,1,0,4,0.0,0.0,0.0,...,4.591702,3.074624,2.827554,2.725723,-7.479535,-7.047246,-5.115048,NaN,NaN,NaN
3,2019-01-04,AG,3,2019,1,0,5,0.0,0.0,0.0,...,6.704284,2.796959,2.596201,3.032581,-5.066425,-4.710205,-4.872170,0.270229,0.251229,0.015164
4,2019-01-05,AG,4,2019,1,0,6,0.0,0.0,0.0,...,5.915830,0.389317,0.361944,-0.126518,-1.979819,-1.840618,-1.831873,0.300774,0.279627,-1.708167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19261,2021-01-18,ZH,736,2021,1,3,1,75.0,210.0,8.0,...,6.205472,3.075791,2.393691,2.023425,-5.492244,-4.795672,-1.306148,-1.623621,-1.509465,0.002693
19262,2021-01-19,ZH,737,2021,1,3,2,75.0,426.0,15.0,...,3.858448,2.718617,2.026134,2.165765,-3.185923,-3.012793,-2.428060,-1.015020,-0.943654,0.000000
19263,2021-01-20,ZH,738,2021,1,3,3,75.0,454.0,8.0,...,2.978518,2.834349,2.264153,2.391853,-2.534243,-2.363485,-3.144802,1.329377,1.235909,0.000000
19264,2021-01-21,ZH,739,2021,1,3,4,75.0,316.0,7.0,...,3.777056,2.103175,1.726089,1.426748,-2.740384,-2.582821,-0.043509,0.487807,0.453509,-2.001956


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
8. Export and re-read the dataframe containing the (average and total) sentiments the cantons' readerships are exposed to
</h2>
</div>

In [82]:
# Export the senti_exposure dataframe as a csv file
senti_exposure.to_csv("Analysis/senti_exposure_by_topic.csv", encoding = 'utf-8-sig')

In [83]:
# Read in the data as follows
senti_exposure = pd.read_csv("Analysis/senti_exposure_by_topic.csv", index_col = 0)
# Transform publication_date into a daily date type
senti_exposure.publication_date = pd.to_datetime(senti_exposure.publication_date, yearfirst = True).astype('datetime64[D]')
# Take a look at the dataframe
senti_exposure

,publication_date,canton,days_elapsed,year,month,week,weekday,stridx,new_conf,new_deaths,...,tot_Blob_polarity_sports,tot_Vader_polarity_adj_tourism,tot_Vader_polarity_adj_2_tourism,tot_Blob_polarity_tourism,tot_Vader_polarity_adj_tragedies_crimes,tot_Vader_polarity_adj_2_tragedies_crimes,tot_Blob_polarity_tragedies_crimes,tot_Vader_polarity_adj_transportation,tot_Vader_polarity_adj_2_transportation,tot_Blob_polarity_transportation
0,2019-01-01,AG,0,2019,1,0,2,0.0,0.0,0.0,...,5.090300,3.591609,3.339084,1.780409,-3.958509,-3.858086,-6.088240,NaN,NaN,NaN
1,2019-01-02,AG,1,2019,1,0,3,0.0,0.0,0.0,...,6.541127,3.561153,3.283226,2.673688,-3.231048,-3.046774,-2.030495,NaN,NaN,NaN
2,2019-01-03,AG,2,2019,1,0,4,0.0,0.0,0.0,...,4.591702,3.074624,2.827554,2.725723,-7.479535,-7.047246,-5.115048,NaN,NaN,NaN
3,2019-01-04,AG,3,2019,1,0,5,0.0,0.0,0.0,...,6.704284,2.796959,2.596201,3.032581,-5.066425,-4.710205,-4.872170,0.270229,0.251229,0.015164
4,2019-01-05,AG,4,2019,1,0,6,0.0,0.0,0.0,...,5.915830,0.389317,0.361944,-0.126518,-1.979819,-1.840618,-1.831873,0.300774,0.279627,-1.708167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19261,2021-01-18,ZH,736,2021,1,3,1,75.0,210.0,8.0,...,6.205472,3.075791,2.393691,2.023425,-5.492244,-4.795672,-1.306148,-1.623621,-1.509465,0.002693
19262,2021-01-19,ZH,737,2021,1,3,2,75.0,426.0,15.0,...,3.858448,2.718617,2.026134,2.165765,-3.185923,-3.012793,-2.428060,-1.015020,-0.943654,0.000000
19263,2021-01-20,ZH,738,2021,1,3,3,75.0,454.0,8.0,...,2.978518,2.834349,2.264153,2.391853,-2.534243,-2.363485,-3.144802,1.329377,1.235909,0.000000
19264,2021-01-21,ZH,739,2021,1,3,4,75.0,316.0,7.0,...,3.777056,2.103175,1.726089,1.426748,-2.740384,-2.582821,-0.043509,0.487807,0.453509,-2.001956
